# 1. Import the required libraries

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))
print(sys.executable)

/Users/jipuiters/Documents/PYTHON/Erasmus/ReSurfEMG/venv/bin/python


In [2]:
# Standard code libraries
import os
import matplotlib.pyplot as plt
import numpy as np

# Custom code libraries from ReSurfEMG
from resurfemg.data_connector.config import Config
from resurfemg.data_connector import file_discovery
from resurfemg.postprocessing import features as feat
from resurfemg.postprocessing import quality_assessment as qa
from resurfemg.pipelines import ipy_widgets
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_connector.data_classes import (
VentilatorDataGroup, EmgDataGroup)

#%matplotlib widget
%matplotlib inline


## 2. Load the ventilator and sEMG data

In [3]:
import os
import re
import numpy as np
from resurfemg.data_connector.data_classes import EmgDataGroup, VentilatorDataGroup

# === Geldige basis tijdstippen ===
valid_base_timepoints = {"t0", "t1", "t2", "t3", "t5", "t24", "t48"}

# === Pad naar je datafolder ===
base_path = os.path.expanduser('~/Documents/Anonymous_data_kopie')
all_files = [f for f in os.listdir(base_path) if f.endswith('.npy')]

# === Aangepaste regex: haal ook versie eruit ===
pattern = re.compile(
    r'(?P<type>two_track_emg|paw)_(?P<id>\d{3})_(?P<tp>t\d+)(?:_v(?P<version>\d+))?(?:_.*)?\.npy'
)

# === Data mapping ===
record_map = {}

for f in all_files:
    if "nodata" in f.lower():
        print(f"⛔ File contains no data and is skipped: {f}")
        continue

    match = pattern.match(f)
    if not match:
        print(f"⚠️  Could not parse name of file: {f}")
        continue

    data_type = match.group("type")
    patient_id = match.group("id")
    base_tp = match.group("tp")
    version = match.group("version")

    # Alleen tijdstippen gebruiken die geldig zijn
    if base_tp not in valid_base_timepoints:
        print(f"⚠️  invalid time iD '{base_tp}' in file {f} — skipped")
        continue

    # Nieuw tijdstiplabel: t1v1 of gewoon t1
    timepoint = base_tp if version is None else f"{base_tp}v{version}"
    key = (patient_id, timepoint)

    if key not in record_map:
        record_map[key] = {}
    if data_type == "two_track_emg":
        record_map[key]["emg"] = f
    elif data_type == "paw":
        record_map[key]["vent"] = f

# === Data inladen ===
patient_data = {}

for (pid, tp), files in sorted(record_map.items()):
    # Meld als er maar één van de twee typen is
    if "emg" not in files and "vent" in files:
        print(f"❌ Only ventilatorfile found patient {pid}, {tp} — EMG file missing. Data not loaded.")
        continue
    elif "vent" not in files and "emg" in files:
        print(f"❌ Only EMG-file found patient {pid}, {tp} — ventilatorfile missing. Data not loaded.")
        continue

    emg_path = os.path.join(base_path, files["emg"])
    vent_path = os.path.join(base_path, files["vent"])

    y_emg = np.load(emg_path)
    y_vent = np.load(vent_path)

    fs_emg = 2048
    fs_vent = 100

    emg_ts = EmgDataGroup(
        y_emg,
        fs=fs_emg,
        labels=["parasternal", "diaphragm"],
        units=2 * ["uV"]
    )

    vent_ts = VentilatorDataGroup(
        y_vent,
        fs=fs_vent,
        labels=["Paw"],
        units=["cmH2O"]
    )

    if pid not in patient_data:
        patient_data[pid] = {}
    patient_data[pid][tp] = {
        "emg": emg_ts,
        "vent": vent_ts,
        "emg_file": emg_path,
        "vent_file": vent_path
    }

# === Samenvatting ===
print(f"\n✅ Complete import: {len(patient_data)} patients loaded.\n")
for pid in sorted(patient_data.keys()):
    print(f"Patient {pid}:")
    for tp in sorted(patient_data[pid].keys(), key=lambda x: int(re.search(r'\d+', x).group())):
        print(f"  Time {tp}:")
        print(f"    EMG-file:   {os.path.basename(patient_data[pid][tp]['emg_file'])}")
        print(f"    Vent-file:  {os.path.basename(patient_data[pid][tp]['vent_file'])}")

⛔ File contains no data and is skipped: paw_012_t1_v1_nodata.npy
⛔ File contains no data and is skipped: paw_006_nodata.npy
⛔ File contains no data and is skipped: two_track_emg_006_nodata.npy
⛔ File contains no data and is skipped: two_track_emg_012_t1_v1_nodata.npy
⛔ File contains no data and is skipped: two_track_emg_005_t1_v1_nodata.npy
⛔ File contains no data and is skipped: two_track_emg_011_t0_v2_nodata.npy
⛔ File contains no data and is skipped: paw_011_t0_v2_nodata.npy
⛔ File contains no data and is skipped: paw_005_t1_v1_nodata.npy
No ECG channel detected. Set ECG channel index with `EmgDataGroup.set_ecg_idx(arg)` method.
Auto-detected Pvent channel from labels.
No ECG channel detected. Set ECG channel index with `EmgDataGroup.set_ecg_idx(arg)` method.
Auto-detected Pvent channel from labels.
No ECG channel detected. Set ECG channel index with `EmgDataGroup.set_ecg_idx(arg)` method.
Auto-detected Pvent channel from labels.
No ECG channel detected. Set ECG channel index with `

# 3. Pre-process the data

In [4]:
# Filter
#emg_timeseries.run('filter_emg')
# Which equals:
# emg_timeseries.run(
#     'filter_emg',
#     signal_type='raw',
#     hp_cf=20.0,
#     lp_cf=500.0,    
#     channel_idxs=[0, 1],
# )
# Where:
# signal_type:      Filter the raw, just assigned, data
# hp_cf:            High-pass cut-off frequency of 20 Hz
# lp_cf:            Low-pass cut-off frequency of 500 Hz
# channel_idxs:     For all channels (None would default to this)

# --> Data is stored in:
# emg_timeseries.channels[channel_idx].y_filt
# Or for short:
# emg_timeseries[channel_idx].y_filt
# Filter alle EMG-data op alle tijdstippen
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        print(f"Applying bandpass filter patient {pid}, {tp}...")
        
        emg_ts = data['emg']
        
        # Voer standaard filtering uit (20-500 Hz bandpass)
        emg_ts.run(
            'filter_emg',
            signal_type='raw',   # filter op de originele data
            hp_cf=20.0,
            lp_cf=500.0,
            channel_idxs=None    # None = alle kanalen
        )
        
        print(f" Filtering complete patient {pid},{tp}.")

print("\n All EMG-signals filtered and stored in `.y_filt` per channel.")



Applying bandpass filter patient 001, t0...


TypeError: emg_bandpass_butter() got an unexpected keyword argument 'fs_emg'

## 3.1 Eliminate ECG

In [ ]:
# # Through gating

# # Gate the EMG
#emg_timeseries.run('gating', overwrite=True)
# # Which equals:
# # emg_timeseries.run(
# #     'gating'
# #     signal_type='filt',        
# #     gate_width_samples=None,    
# #     ecg_peak_idxs=None,         
# #     ecg_raw=None,               
# #     bp_filter=True,
# #     channel_idxs=None,
# #     overwrite=True)
# # )
# # Where:
# # signal_type:          Filter the clean, just filtered, data
# # gate_width_samples:   Gate width, `None` defaults to fs // 10
# # ecg_peak_idxs:        Sample idxs of ECG peaks, when `None` peaks are 
# #                       automatically identified.
# # ecg_raw:              ECG data to detect ECG peaks in if no ecg_peak_idxs are
# #                       provided. If `None` and no ecg-channel is detected
# #                       from the labels the raw channel data is used.
# # bp_filter:            True/False: Filter the provided ecg_raw between 1-500
# #                       Hz before peak detection
# # channel_idxs:         For all channels (None would default to this)
# # overwrite:            Overwrite the existing ECG peaks

# # --> Data is stored in:
# # emg_timeseries.channels[channel_idx].y_clean
# # Or for short:
# # emg_timeseries[channel_idx].y_clean

# Verwijder ECG-artefacten via gating voor alle tijdstippen
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        print(f" Removing ECG-Artefacts patient {pid},{tp}...")
        
        emg_ts = data['emg']
        
        emg_ts.run(
            'gating',
            signal_type='filt',  # gebruik gefilterde data
            gate_width_samples=int(emg_ts.param['fs'] * 0.40),  # 400 ms
            ecg_peak_idxs=None,
            ecg_raw=None,
            bp_filter=True,
            channel_idxs=None,
            overwrite=True
        )
        
        print(f"Artefacts succesfully removed patient {pid}, {tp}.")

print("\n All ECG-artefacts removed and stored in `.y_clean` per channel.")


In [ ]:
# Calculate the envelope of the signal
#emg_timeseries.run('envelope')
# Which equals:
# emg_timeseries.run(
#     'envelope',
#     env_window=None,
#     env_type='rms',
#     signal_type='clean',
# )
# Where:
# env_window:           Envelope window width, `None` defaults to fs // 5
# env_type:             'rms' for root-mean-square (default), 'arv' for average
#                       rectified
# signal_type:          Calculate the envelope over the clean data

# --> Data is stored in:
# emg_timeseries.channels[channel_idx].y_env
# Or for short:
# emg_timeseries[channel_idx].y_env

# Envelope berekenen over het 'clean' signaal (na filtering & gating)
# Bereken envelopes voor alle patiënten en tijdstippen
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        print(f"Applying envelope patient {pid}, {tp}...")

        emg_ts = data['emg']

        emg_ts.run(
            'envelope',
            env_window= 3*(fs_emg // 5),   # standaard = fs // 5, nu dus 3*breder dan default= 600 ms
            env_type='rms',          # of 'arv' voor average rectified
            signal_type='clean'      # gebruik gated signaal
        )

        print(f"succesfully calculated Envelope patient {pid}, {tp}.")

print("\nEnvelopes stored in `.y_env` per channel.")



In [ ]:
# Calculate the baseline for the EMG envelopes and p_vent
#emg_timeseries.run('baseline')
# Which equals:
# emg_timeseries.run(
#    'baseline',
#     percentile=33,
#     window_s=int(7.5 * fs_emg),
#     step_s=fs_emg // 5,
#     method='default',
#     signal_type='env',
#     augm_percentile=25,
#     ma_window=None,
#     perc_window=None,
#     channel_idxs=[0, 1], 
# )
# Where:
# percentile:       Percentile of signal in the window to take as the baseline
#                   value 
# window_s:         Window length in samples (default: int(7.5 * fs))
# step_s:           Number of consecutive samples with the same baseline value
#                   (default: fs // 5)
# method:           'default' or 'slopesum'
# signal_type:      Calculate the baseline over the envelope (y_env) for emg 
#                   and over the original signal for p_vent
# augm_percentile   Augmented_percentile for slopesum baseline
# ma_window:        Moving average window in samples for average dy/dt in
#                   slopesum baseline
# perc_window:      'perc_window' for slopesum baseline.
# channel_idxs:     For all channels (None would default to this)

# --> Data is stored in:
# emg_timeseries.channels[channel_idx].y_baseline

#vent_timeseries.run(
#    'baseline',
#    channel_idxs=[0],
#    signal_type='raw')

# Baseline berekenen voor zowel EMG (env) als ventilatordruk (raw)
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        print(f"calculating baseline patient {pid}, {tp}...")

        emg_ts = data['emg']
        vent_ts = data['vent']

        # EMG baseline op de envelope
        emg_ts.run(
            'baseline',
            signal_type='env',     # werk op de RMS-envelope
            channel_idxs=None      # beide leads
        )

        # Ventilator-baseline op het ruwe signaal
        vent_ts.run(
            'baseline',
            signal_type='raw',
            channel_idxs=[0]
        )

        print(f"Baseline ready patient {pid}, {tp}.")

print("\nBaselines stored in `.y_baseline` per channel.")

In [ ]:
# MOTION ARTEFACT DETECTION
import numpy as np
import csv

def calculate_baseline_drift(baseline, window_size):
    """
    Calculate baseline drift: Divide the baseline signal into non-overlapping windows,
    and for each window compute (max - min).
    """
    drift = []
    for i in range(0, len(baseline) - window_size + 1, window_size):
        window = baseline[i:i + window_size]
        drift.append(max(window) - min(window))
    return drift

def identify_abnormal_windows(drift, window_size, fs, threshold_factor=3):
    """
    Identify abnormal windows using the median absolute deviation (MAD).
    Returns a list of tuples with (start_time, end_time) for windows that are abnormal.
    """
    median_drift = np.median(drift)
    mad = np.median(np.abs(drift - median_drift))
    threshold = median_drift + threshold_factor * mad
    abnormal_windows = [i for i, d in enumerate(drift) if d > threshold]

    abnormal_times = []
    if abnormal_windows:
        start_time = abnormal_windows[0] * (window_size / fs)
        end_time = start_time + (window_size / fs)
        for i in range(1, len(abnormal_windows)):
            current_time = abnormal_windows[i] * (window_size / fs)
            if current_time <= end_time:  # overlapping or adjacent
                end_time += (window_size / fs)
            else:
                abnormal_times.append((start_time, end_time))
                start_time = current_time
                end_time = start_time + (window_size / fs)
        abnormal_times.append((start_time, end_time))
    return abnormal_times

def merge_time_windows(time_windows):
    """
    Merge overlapping or adjacent time windows.
    """
    if not time_windows:
        return []
    time_windows.sort()  # sort by start time
    merged_windows = [time_windows[0]]
    for current_start, current_end in time_windows[1:]:
        last_start, last_end = merged_windows[-1]
        if current_start <= last_end:  # overlapping or adjacent
            merged_windows[-1] = (last_start, max(last_end, current_end))
        else:
            merged_windows.append((current_start, current_end))
    return merged_windows

#############################################
# PRINTING FUNCTION WITH SEPARATE ABNORMAL TIME WINDOWS
#############################################

def print_formatted_results(pid, tp, emg_baseline_drift, vent_baseline_drift, filtered_channels, ab_paras, ab_diaphragm, ab_vent):
    print(f"\nResults for Patient {pid}, Time {tp}:")
    
    # Parasternal results (lead index 0)
    print(f"  Parasternal: Mean Baseline Drift({np.mean(emg_baseline_drift[0]):.2f}), Median Baseline Drift({np.median(emg_baseline_drift[0]):.2f}), Max Baseline Drift({np.max(emg_baseline_drift[0]):.2f})")
    if ab_paras == "filtered_out":
        print("    Abnormal Time Windows Parasternal: Whole signal filtered out")
    elif ab_paras:
        merged = merge_time_windows(ab_paras)
        print(f"    Abnormal Time Windows Parasternal: {merged}")
    else:
        print("    Abnormal Time Windows Parasternal: None")

    # Diaphragm results (lead index 1)
    print(f"  Diaphragm: Mean Baseline Drift({np.mean(emg_baseline_drift[1]):.2f}), Median Baseline Drift({np.median(emg_baseline_drift[1]):.2f}), Max Baseline Drift({np.max(emg_baseline_drift[1]):.2f})")
    if ab_diaphragm == "filtered_out":
        print("    Abnormal Time Windows Diaphragm: Whole signal filtered out")
    elif ab_diaphragm:
        merged = merge_time_windows(ab_diaphragm)
        print(f"    Abnormal Time Windows Diaphragm: {merged}")
    else:
        print("    Abnormal Time Windows Diaphragm: None")

    # Ventilator results (assumed one channel for vent)
    print(f"  Ventilator: Mean Baseline Drift({np.mean(vent_baseline_drift[0]):.2f}), Median Baseline Drift({np.median(vent_baseline_drift[0]):.2f}), Max Baseline Drift({np.max(vent_baseline_drift[0]):.2f})")
    if ab_vent == "filtered_out":
        print("    Abnormal Time Windows Ventilator: Whole signal filtered out")
    elif ab_vent:
        merged = merge_time_windows(ab_vent)
        print(f"    Abnormal Time Windows Ventilator: {merged}")
    else:
        print("    Abnormal Time Windows Ventilator: None")
    
    if filtered_channels:
        for channel in filtered_channels:
            print(f"  Note: Measurement for patient {pid}, time {tp}, channel {channel} filtered out due to high median baseline drift.")

#############################################
# MAIN SCRIPT: ITERATE OVER PATIENT DATA AND WRITE CSV OUTPUT
#############################################

# Settings
fs_emg = 2048  
fs_ventilator = 100  
window_size_emg = int(5 * fs_emg)  
window_size_vent = int(5 * fs_ventilator)  
median_drift_threshold = 5.0  
mad_threshold_factor = 8.0  

channel_names = ["parasternal", "diaphragm"]

# Dictionary to store abnormal times for each patient and time point.
# Structure: abnormal_times_dict[pid][tp] = {"parasternal": ..., "diaphragm": ..., "ventilator": ...}
abnormal_times_dict = {}

# List to store CSV rows.
# We create separate columns for each lead's drift stats and abnormal time windows.
csv_rows = []

for pid, patient in patient_data.items():
    for tp, data in patient.items():
        emg_ts = data['emg']
        vent_ts = data['vent']

        # Initialize abnormal times dictionary for this patient and time if not present.
        if pid not in abnormal_times_dict:
            abnormal_times_dict[pid] = {}
        if tp not in abnormal_times_dict[pid]:
            abnormal_times_dict[pid][tp] = {}

        # To store drift and abnormal time windows for each lead.
        emg_baseline_drift = []
        vent_baseline_drift = []
        filtered_channels = []  # To note which channels are fully filtered out.

        # Process EMG channels separately.
        for channel_idx in range(len(emg_ts.channels)):
            baseline = emg_ts.channels[channel_idx].y_baseline
            drift = calculate_baseline_drift(baseline, window_size_emg)
            emg_baseline_drift.append(drift)
            median_current = np.median(drift)
            # Filter based on high median baseline drift instead of the mean.
            if median_current > median_drift_threshold:
                filtered_channels.append(channel_names[channel_idx])
                abnormal_times = "filtered_out"
            else:
                abnormal_times = identify_abnormal_windows(drift, window_size_emg, fs_emg, mad_threshold_factor)
            # Store the abnormal times for this EMG channel using the lead name as key.
            abnormal_times_dict[pid][tp][channel_names[channel_idx]] = abnormal_times

        # Process Ventilator channels.
        for channel_idx in range(len(vent_ts.channels)):
            baseline = vent_ts.channels[channel_idx].y_baseline
            drift = calculate_baseline_drift(baseline, window_size_vent)
            vent_baseline_drift.append(drift)
            median_current = np.median(drift)
            if median_current > median_drift_threshold:
                filtered_channels.append(f"Ventilator channel {channel_idx}")
                abnormal_times_v = "filtered_out"
            else:
                abnormal_times_v = identify_abnormal_windows(drift, window_size_vent, fs_ventilator, mad_threshold_factor)
            # For the ventilator, we assume one channel is used for the summary.
            if channel_idx == 0:
                abnormal_times_dict[pid][tp]["ventilator"] = abnormal_times_v

        # Create CSV row with separate columns.
        # We report drift values (mean, median and max) for each lead and then the abnormal time windows.
        row = [
            pid,
            tp,
            f"Parasternal: Mean Drift: {np.mean(emg_baseline_drift[0]):.2f}, Median Drift: {np.median(emg_baseline_drift[0]):.2f}, Max Drift: {np.max(emg_baseline_drift[0]):.2f}",
            f"Abnormal Parasternal: {abnormal_times_dict[pid][tp].get('parasternal', None)}",
            f"Diaphragm: Mean Drift: {np.mean(emg_baseline_drift[1]):.2f}, Median Drift: {np.median(emg_baseline_drift[1]):.2f}, Max Drift: {np.max(emg_baseline_drift[1]):.2f}",
            f"Abnormal Diaphragm: {abnormal_times_dict[pid][tp].get('diaphragm', None)}",
            f"Ventilator: Mean Drift: {np.mean(vent_baseline_drift[0]):.2f}, Median Drift: {np.median(vent_baseline_drift[0]):.2f}, Max Drift: {np.max(vent_baseline_drift[0]):.2f}",
            f"Abnormal Ventilator: {abnormal_times_dict[pid][tp].get('ventilator', None)}",
            f"Filtered Channels: {filtered_channels}"
        ]
        csv_rows.append(row)

        # Retrieve separate abnormal times for printing.
        ab_paras = abnormal_times_dict[pid][tp].get("parasternal", None)
        ab_diaphragm = abnormal_times_dict[pid][tp].get("diaphragm", None)
        ab_vent = abnormal_times_dict[pid][tp].get("ventilator", None)
        print_formatted_results(pid, tp, emg_baseline_drift, vent_baseline_drift, filtered_channels, ab_paras, ab_diaphragm, ab_vent)

print("\nAverage, median and maximum baseline drift measurements printed for each channel.")

# Save the output to a CSV file with separate columns for each abnormal windows field.
with open('baseline_drift_analysis.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    headers = [
        "Patient ID", "Time ID",
        "Parasternal", "Abnormal Parasternal",
        "Diaphragm", "Abnormal Diaphragm",
        "Ventilator", "Abnormal Ventilator",
        "Filtered Channels"
    ]
    csv_writer.writerow(headers)
    csv_writer.writerows(csv_rows)

print("Output saved to baseline_drift_analysis.csv")



In [ ]:
#VISUAL INSPECTION MOTION ARTEFACT EPOCHS
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import numpy as np
import csv
import ast

#############################################
# STEP 1: LOAD ABNORMAL TIME WINDOWS FROM CSV
#############################################

def load_abnormal_times(csv_filename):
    """
    Loads the CSV file and parses the abnormal time window fields for each patient and time.
    Returns a dictionary with structure:
       abnormal[patient_id][time_id] = {
           "parasternal": [ (start, end), ... ] or "filtered_out",
           "diaphragm":   [ (start, end), ... ] or "filtered_out",
           "ventilator":  [ (start, end), ... ] or "filtered_out"
       }
    The CSV file contains columns such as:
      "Abnormal Parasternal", "Abnormal Diaphragm", "Abnormal Ventilator"
    Each value is of the form:
      "Abnormal Parasternal: [(0.0, 15.0), (30.0, 35.0), ...]"
    """
    abnormal = {}
    with open(csv_filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            pid = row["Patient ID"]
            tp = row["Time ID"]
            if pid not in abnormal:
                abnormal[pid] = {}
            abnormal[pid][tp] = {}
            
            # Parse the abnormal windows for each channel
            # For Parasternal:
            ap_str = row["Abnormal Parasternal"].strip()
            if ap_str.startswith("Abnormal Parasternal:"):
                ap_str = ap_str.split(":", 1)[1].strip()
            if ap_str.lower() == "filtered_out":
                abnormal[pid][tp]["parasternal"] = "filtered_out"
            else:
                try:
                    abnormal[pid][tp]["parasternal"] = ast.literal_eval(ap_str)
                except Exception as e:
                    abnormal[pid][tp]["parasternal"] = []
                    
            # For Diaphragm:
            ad_str = row["Abnormal Diaphragm"].strip()
            if ad_str.startswith("Abnormal Diaphragm:"):
                ad_str = ad_str.split(":", 1)[1].strip()
            if ad_str.lower() == "filtered_out":
                abnormal[pid][tp]["diaphragm"] = "filtered_out"
            else:
                try:
                    abnormal[pid][tp]["diaphragm"] = ast.literal_eval(ad_str)
                except Exception as e:
                    abnormal[pid][tp]["diaphragm"] = []
            
            # For Ventilator:
            av_str = row["Abnormal Ventilator"].strip()
            if av_str.startswith("Abnormal Ventilator:"):
                av_str = av_str.split(":", 1)[1].strip()
            if av_str.lower() == "filtered_out":
                abnormal[pid][tp]["ventilator"] = "filtered_out"
            else:
                try:
                    abnormal[pid][tp]["ventilator"] = ast.literal_eval(av_str)
                except Exception as e:
                    abnormal[pid][tp]["ventilator"] = []
    return abnormal

# Load the CSV file (make sure the file name/path is correct)
abnormal_times_dict = load_abnormal_times("baseline_drift_analysis.csv")


#############################################
# STEP 2: DEFINE A HELPER FOR PLOTTING ABNORMAL WINDOWS
#############################################

def plot_abnormal_windows(ax, abnormal_windows, t_start, t_end):
    """
    Overlays a translucent red region on the axis 'ax' from t_start to t_end for
    every abnormal time window that overlaps with the current display.
    
    If abnormal_windows is "filtered_out", shade the whole (t_start, t_end) region.
    """
    if abnormal_windows == "filtered_out":
        ax.axvspan(t_start, t_end, color='red', alpha=0.3)
    elif abnormal_windows and isinstance(abnormal_windows, list):
        for (win_start, win_end) in abnormal_windows:
            # Only mark the overlapping portion of any abnormal window 
            if win_end > t_start and win_start < t_end:
                ax.axvspan(max(t_start, win_start), min(t_end, win_end), color='red', alpha=0.3)


#############################################
# STEP 3: MODIFY THE INTERACTIVE PLOTTING FUNCTION
#############################################

def scrollable_emg_plot(pid, tp, window_size=10, y_lims_emg=None, y_lims_vent=None):
    """
    Displays an interactive plot of the EMG (parasternal, diaphragm) envelope 
    and baseline plus the ventilator signal for a given patient and time.
    Overlays red shading for low‐quality segments as defined from the CSV.
    
    The CSV abnormal times are loaded into abnormal_times_dict, where each 
    channel is assigned to one of: "parasternal", "diaphragm", or "ventilator".
    """
    if pid not in patient_data or tp not in patient_data[pid]:
        print(f"❌ Invalid patient ID or Time ID ({pid}, {tp})")
        return
    
    emg_ts = patient_data[pid][tp]['emg']
    vent_ts = patient_data[pid][tp]['vent']
    
    # Retrieve abnormal windows for this patient and time (if available).  
    # This dictionary should now have keys "parasternal", "diaphragm", "ventilator".
    abnormal = abnormal_times_dict.get(pid, {}).get(tp, {})

    # Determine the global time axis from the first EMG channel:
    t_data = emg_ts[0].t_data
    t_min = t_data[0]
    t_max = t_data[-1]
    t_slider_max = t_max - window_size

    @interact(t_start=FloatSlider(min=t_min, max=t_slider_max, step=1, value=t_min, description='Start (s)'))
    def update_plot(t_start):
        t_end = t_start + window_size

        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 8), sharex=True)
        axes_emg = axes[:2]
        ax_vent = axes[2]

        colors_clean = ['lightgray', 'lightgray']
        colors_env = ['tab:blue', 'tab:green']

        # Plot the cleaned (filtered) EMG signals.
        emg_ts.run('plot_full', axes=axes_emg, signal_type='clean', colors=colors_clean, baseline_bool=False)

        # Add envelope and baseline for each EMG channel.
        for ch_idx, ax in enumerate(axes_emg):
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_env, color=colors_env[ch_idx],
                    linewidth=2.0, label='envelope')
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_baseline, color=colors_env[ch_idx],
                    linestyle='--', linewidth=1.5, label='baseline')
            ax.legend(loc='upper right')
        axes_emg[0].set_title(f'EMG - patient {pid}, {tp}')
        axes_emg[-1].set_xlabel('Time (s)')

        # Plot the ventilator signal.
        vent_ts.run('plot_full', axes=[ax_vent])
        ax_vent.set_title(f'Ventilator - patient {pid}, {tp}')
        ax_vent.set_xlabel('Time (s)')

        # Set x-axis limits for all subplots.
        for ax in axes:
            ax.set_xlim([t_start, t_end])
        if y_lims_emg is not None:
            for ax in axes_emg:
                ax.set_ylim(y_lims_emg)
        if y_lims_vent is not None:
            ax_vent.set_ylim(y_lims_vent)

        # Get abnormal time windows per channel
        ab_paras = abnormal.get("parasternal", [])
        ab_diaphragm = abnormal.get("diaphragm", [])
        ab_vent = abnormal.get("ventilator", [])

        # Overlay red shading for each channel.
        plot_abnormal_windows(axes_emg[0], ab_paras, t_start, t_end)
        plot_abnormal_windows(axes_emg[1], ab_diaphragm, t_start, t_end)
        plot_abnormal_windows(ax_vent, ab_vent, t_start, t_end)

        plt.tight_layout()
        plt.show()

#############################################
# STEP 4: USE THE INTERACTIVE PLOT
#############################################

# (Ensure that patient_data is defined and loaded as in your original scripts.)
scrollable_emg_plot(
    pid='005',       # example patient id
    tp='t0',         # example time id
    window_size=40,  # time window in seconds
    y_lims_emg=[-5, 30],
    y_lims_vent=[-5, 25]
)


In [ ]:
#BACKGROUND NOISE DETECTION
import numpy as np
import csv

def calculate_drift(signal, window_size):
    """
    Divide the signal into non-overlapping windows and calculate the drift (max - min)
    for each window.
    """
    drift = []
    for i in range(0, len(signal) - window_size + 1, window_size):
        window = signal[i : i + window_size]
        drift.append(max(window) - min(window))
    return drift

def print_formatted_results(pid, tp, emg_stats, vent_stats, measurement_status):
    """
    Print summary statistics for each channel and the overall measurement status.
    """
    print(f"\nResults for Patient {pid}, Time {tp}:")
    print(f"  Parasternal (Envelope): Mean Drift: {emg_stats[0][0]:.2f}, Median Drift: {emg_stats[0][1]:.2f}, Max Drift: {emg_stats[0][2]:.2f}")
    print(f"  Diaphragm (Envelope):  Mean Drift: {emg_stats[1][0]:.2f}, Median Drift: {emg_stats[1][1]:.2f}, Max Drift: {emg_stats[1][2]:.2f}")
    print(f"  Ventilator (Raw):      Mean Drift: {vent_stats[0]:.2f}, Median Drift: {vent_stats[1]:.2f}, Max Drift: {vent_stats[2]:.2f}")
    print(f"  Measurement Status: {measurement_status}")

#############################################
# MAIN SCRIPT: DETECTING BACKGROUND NOISE AND SPECIFIC CONDITIONS
#############################################

# Adjustable parameter: window duration in seconds (change this to adjust window size easily)
window_sec = 10  

# Set thresholds
threshold_emg = 4.0    # for EMG channels in uV
threshold_vent = 4.0   # for the ventilator channel in cmH20

# Settings for EMG processing.
fs_emg = 2048  
window_size_emg = int(window_sec * fs_emg)  # window size for EMG envelope data in samples

# Prepare CSV output.
csv_rows = []
headers = [
    "Patient ID", "Time ID",
    "Parasternal (Mean, Median, Max)",
    "Diaphragm (Mean, Median, Max)",
    "Ventilator (Mean, Median, Max)",
    "Measurement Status"
]

# Loop across patient measurements.
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        emg_ts = data['emg']
        vent_ts = data['vent']  # ventilator data group

        # Process EMG channels using envelope data (y_env).
        emg_stats = []
        for channel_idx in range(len(emg_ts.channels)):
            envelope = emg_ts.channels[channel_idx].y_env
            drift = calculate_drift(envelope, window_size_emg)
            mean_drift = np.mean(drift)
            median_drift = np.median(drift)
            max_drift = np.max(drift)
            emg_stats.append((mean_drift, median_drift, max_drift))
        
        # Process the Ventilator channel using its raw data.
        vent_channel = vent_ts.channels[0]        # assuming a single ventilator channel
        fs_vent = vent_ts.param['fs']              # obtain ventilator sampling frequency (e.g. 100)
        window_size_vent = int(window_sec * fs_vent) # window size in samples for ventilator
        raw_data = vent_channel.y_raw              # raw ventilator data using y_raw
        drift_vent = calculate_drift(raw_data, window_size_vent)
        vent_mean = np.mean(drift_vent)
        vent_median = np.median(drift_vent)
        vent_max = np.max(drift_vent)
        vent_stats = (vent_mean, vent_median, vent_max)
        
        # Determine measurement status based on channel-specific thresholds.
        status_list = []
        if vent_median <= threshold_vent:
            status_list.append("ventilator malfunction")
        if emg_stats[1][1] <= threshold_emg:  # channel index 1: diaphragm
            status_list.append("no respiratory activity diaphragm")
        if emg_stats[0][1] <= threshold_emg:  # channel index 0: parasternal
            status_list.append("no respiratory activity parasternal")
        if not status_list:
            status_list.append("respiratory activity")
        measurement_status = ", ".join(status_list)

        # Print the results.
        print_formatted_results(pid, tp, emg_stats, vent_stats, measurement_status)
        
        # Prepare CSV row.
        row = [
            pid,
            tp,
            f"Mean: {emg_stats[0][0]:.2f}, Median: {emg_stats[0][1]:.2f}, Max: {emg_stats[0][2]:.2f}",
            f"Mean: {emg_stats[1][0]:.2f}, Median: {emg_stats[1][1]:.2f}, Max: {emg_stats[1][2]:.2f}",
            f"Mean: {vent_stats[0]:.2f}, Median: {vent_stats[1]:.2f}, Max: {vent_stats[2]:.2f}",
            measurement_status
        ]
        csv_rows.append(row)

# Save results to a CSV file.
with open('background_noise_detection.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(headers)
    csv_writer.writerows(csv_rows)

print("\nCSV output saved to background_noise_detection.csv")



In [ ]:
#ASSIGNING LABEL TO MEASUREMENTS, INCLUDE, EXCLUDE OR SUBANALYSIS
import csv
import ast

# File paths
background_csv = 'background_noise_detection.csv'
baseline_csv = 'baseline_drift_analysis.csv'
output_csv = 'measurement_classification.csv'

# Load background noise detection
bg_data = {}
with open(background_csv, newline='') as bg_file:
    reader = csv.DictReader(bg_file)
    for row in reader:
        key = (row["Patient ID"].strip(), row["Time ID"].strip())
        bg_data[key] = row

# Load baseline drift analysis
bd_data = {}
with open(baseline_csv, newline='') as bd_file:
    reader = csv.DictReader(bd_file)
    for row in reader:
        key = (row["Patient ID"].strip(), row["Time ID"].strip())
        bd_data[key] = row

# Combined set of keys
all_keys = set(bg_data.keys()).union(set(bd_data.keys()))

results = []

for key in sorted(all_keys):
    pid, tid = key
    reasons = []
    label = "Include"  # default
    status = bg_data.get(key, {}).get("Measurement Status", "missing").strip().lower()
    raw_filtered = bd_data.get(key, {}).get("Filtered Channels", "").strip()

    # Get readable status from the background noise CSV
    status_str = bg_data.get(key, {}).get("Measurement Status", "").strip()

    # Determine filtered channels list
    if raw_filtered.startswith("Filtered Channels:"):
        filtered_str = raw_filtered[len("Filtered Channels:"):].strip()
    else:
        filtered_str = raw_filtered

    try:
        filtered_list = ast.literal_eval(filtered_str) if filtered_str else []
    except:
        filtered_list = filtered_str

    # Normalize to a list if not already
    if not isinstance(filtered_list, list):
        filtered_list = [filtered_list] if filtered_list else []

    # Check for ventilator malfunction
    if "ventilator malfunction" in status:
        label = "Exclude"
        reasons.append(status_str)

    # Check for background noise: if status is not "respiratory activity", add it as a reason.
    elif status != "respiratory activity":
        reasons.append(status_str)

    # Check for motion artefacts
    if filtered_list:
        fa_reasons = [f"motion artefacts {ch.capitalize()}" for ch in filtered_list]
        reasons.extend(fa_reasons)

    # Exclude if both channels have no respiratory activity.
    # Use substring matching on each reason (splitting combined strings)
    if (any("no respiratory activity diaphragm" in r.lower() for r in reasons) and 
        any("no respiratory activity parasternal" in r.lower() for r in reasons)):
        label = "Exclude"
        # Reset reasons to the two canonical reasons.
        reasons = ["no respiratory activity diaphragm", "no respiratory activity parasternal"]

    # Determine final label (only assess "Subanalysis" when not already Excluded)
    elif not reasons:
        label = "Include"
    elif label != "Exclude":
        motion_only = all("motion artefacts" in r.lower() for r in reasons)
        noise_only = all("no respiratory activity" in r.lower() for r in reasons)
        if len(reasons) == 1 and (motion_only or noise_only):
            label = "Subanalysis"
        else:
            label = "Exclude"

    results.append({
        "Patient ID": pid,
        "Time ID": tid,
        "Status": status_str,
        "Filtered Channels": filtered_list,
        "Label": label,
        "Reason": "; ".join(reasons) if reasons else ""
    })

# Write output CSV
with open(output_csv, mode='w', newline='') as out_file:
    fieldnames = ["Patient ID", "Time ID", "Status", "Filtered Channels", "Label", "Reason"]
    writer = csv.DictWriter(out_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"\nCSV summary saved as {output_csv}")

# Print summary of measurement classification.
total = len(results)
included = sum(1 for r in results if r["Label"] == "Include")
excluded = sum(1 for r in results if r["Label"] == "Exclude")
subanalysis = sum(1 for r in results if r["Label"] == "Subanalysis")

print(f"\nSummary for {total} measurements:")
print(f"  Included     : {included} ({included / total:.2%})")
print(f"  Subanalysis  : {subanalysis} ({subanalysis / total:.2%})")
print(f"  Excluded     : {excluded} ({excluded / total:.2%})")

# Print detailed results
def print_sorted_results(label_name, label_value, show_reason=False):
    print(f"\n{label_name} measurements:")
    filtered = sorted([r for r in results if r["Label"] == label_value], key=lambda x: (x["Patient ID"], x["Time ID"]))
    for r in filtered:
        base_str = f"  {r['Patient ID']} - {r['Time ID']}"
        if show_reason:
            print(f"{base_str}: {r['Reason']}")
        else:
            print(base_str)

print_sorted_results("Included", "Include")
print_sorted_results("Subanalysis", "Subanalysis", show_reason=True)
print_sorted_results("Excluded", "Exclude", show_reason=True)

# --- Additional printed summary for excluded measurements breakdown ---
# We define the following mutually exclusive categories for exclusions:
# 1. "ventilator malfunction": if the measurement reasons contain "ventilator malfunction" (regardless of other reasons).
# 2. "no respiratory activity in EMG channels": if the reasons contain both "no respiratory activity diaphragm" AND "no respiratory activity parasternal".
# 3. "Motion artefacte diaphragm, Background noise parasternal": if the reasons contain both "no respiratory activity parasternal" and "motion artefacts diaphragm".
# 4. "Motion artefacte parasternal, Background noise diaphragm": if the reasons contain both "no respiratory activity diaphragm" and "motion artefacts parasternal".
category_counts = {
    "ventilator malfunction": 0,
    "no respiratory activity in EMG channels": 0,
    "Motion artefacts diaphragm, Background noise parasternal": 0,
    "Motion artefacts parasternal, Background noise diaphragm": 0
}

# Loop over excluded measurements and assign a category.
for r in results:
    if r["Label"] != "Exclude":
        continue
    # Split the reasons string by ";" to get an array and strip each part.
    r_list = [x.strip().lower() for x in r["Reason"].split(";") if x.strip()]
    
    # Check for "ventilator malfunction" first.
    if any("ventilator malfunction" in s for s in r_list):
        category = "ventilator malfunction"
    elif (any("no respiratory activity diaphragm" in s for s in r_list) and 
          any("no respiratory activity parasternal" in s for s in r_list)):
        category = "no respiratory activity in EMG channels"
    elif (any("no respiratory activity parasternal" in s for s in r_list) and 
          any("motion artefacts diaphragm" in s for s in r_list)):
        category = "Motion artefacts diaphragm, Background noise parasternal"
    elif (any("no respiratory activity diaphragm" in s for s in r_list) and 
          any("motion artefacts parasternal" in s for s in r_list)):
        category = "Motion artefacts parasternal, Background noise diaphragm"
    else:
        # In theory, all exclusions should fall into one category.
        category = "UNCLASSIFIED"
        print(f"Unclassified measurement: {r['Patient ID']} - {r['Time ID']} with reasons: {r_list}")
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

print("\n=== Excluded Measurements Breakdown by Category ===")
for cat, count in category_counts.items():
    percentage = (count / total) * 100 if total > 0 else 0
    print(f"{cat}: {count} measurements ({percentage:.2f}% of total measurements)")


In [ ]:
#ASSIGNING LABEL TO MEASUREMENTS, INCLUDE, EXCLUDE OR SUBANALYSIS
import csv
import ast
import pandas as pd

# File paths
background_csv = 'background_noise_detection.csv'
baseline_csv = 'baseline_drift_analysis.csv'
output_csv = 'measurement_classification.csv'

# Load background noise detection
bg_data = {}
with open(background_csv, newline='') as bg_file:
    reader = csv.DictReader(bg_file)
    for row in reader:
        key = (row["Patient ID"].strip(), row["Time ID"].strip())
        bg_data[key] = row

# Load baseline drift analysis
bd_data = {}
with open(baseline_csv, newline='') as bd_file:
    reader = csv.DictReader(bd_file)
    for row in reader:
        key = (row["Patient ID"].strip(), row["Time ID"].strip())
        bd_data[key] = row

# Combined set of keys from both CSVs.
all_keys = set(bg_data.keys()).union(set(bd_data.keys()))

results = []

for key in sorted(all_keys):
    pid, tid = key
    reasons = []
    label = "Include"  # default
    status = bg_data.get(key, {}).get("Measurement Status", "missing").strip().lower()
    raw_filtered = bd_data.get(key, {}).get("Filtered Channels", "").strip()

    # Get readable status from the background noise CSV
    status_str = bg_data.get(key, {}).get("Measurement Status", "").strip()

    # Determine filtered channels list; remove prefixed text if present.
    if raw_filtered.startswith("Filtered Channels:"):
        filtered_str = raw_filtered[len("Filtered Channels:"):].strip()
    else:
        filtered_str = raw_filtered

    try:
        filtered_list = ast.literal_eval(filtered_str) if filtered_str else []
    except:
        filtered_list = filtered_str

    # Normalize to list if needed
    if not isinstance(filtered_list, list):
        filtered_list = [filtered_list] if filtered_list else []

    # Check for ventilator malfunction
    if "ventilator malfunction" in status:
        label = "Exclude"
        reasons.append(status_str)

    # Check for background noise: if status is not "respiratory activity", add it as a reason.
    elif status != "respiratory activity":
        reasons.append(status_str)

    # Check for motion artefacts.
    if filtered_list:
        fa_reasons = [f"motion artefacts {ch.capitalize()}" for ch in filtered_list]
        reasons.extend(fa_reasons)

    # Exclude if both channels have no respiratory activity.
    if (any("no respiratory activity diaphragm" in r.lower() for r in reasons) and 
        any("no respiratory activity parasternal" in r.lower() for r in reasons)):
        label = "Exclude"
        # Reset reasons to the two canonical messages.
        reasons = ["no respiratory activity diaphragm", "no respiratory activity parasternal"]

    # Determine final label (only assess "Subanalysis" when not already Excluded)
    elif not reasons:
        label = "Include"
    elif label != "Exclude":
        motion_only = all("motion artefacts" in r.lower() for r in reasons)
        noise_only = all("no respiratory activity" in r.lower() for r in reasons)
        if len(reasons) == 1 and (motion_only or noise_only):
            label = "Subanalysis"
        else:
            label = "Exclude"

    results.append({
        "Patient ID": pid,
        "Time ID": tid,
        "Status": status_str,
        "Filtered Channels": filtered_list,
        "Label": label,
        "Reason": "; ".join(reasons) if reasons else ""
    })

# Write output CSV
with open(output_csv, mode='w', newline='') as out_file:
    fieldnames = ["Patient ID", "Time ID", "Status", "Filtered Channels", "Label", "Reason"]
    writer = csv.DictWriter(out_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"\nCSV summary saved as {output_csv}")

# Print summary of measurement classification.
total = len(results)
included = sum(1 for r in results if r["Label"] == "Include")
excluded = sum(1 for r in results if r["Label"] == "Exclude")
subanalysis = sum(1 for r in results if r["Label"] == "Subanalysis")

print(f"\nSummary for {total} measurements:")
print(f"  Included     : {included} ({included / total:.2%})")
print(f"  Subanalysis  : {subanalysis} ({subanalysis / total:.2%})")
print(f"  Excluded     : {excluded} ({excluded / total:.2%})")

# Print detailed sorted results.
def print_sorted_results(label_name, label_value, show_reason=False):
    print(f"\n{label_name} measurements:")
    filtered = sorted([r for r in results if r["Label"] == label_value], key=lambda x: (x["Patient ID"], x["Time ID"]))
    for r in filtered:
        base_str = f"  {r['Patient ID']} - {r['Time ID']}"
        if show_reason:
            print(f"{base_str}: {r['Reason']}")
        else:
            print(base_str)

print_sorted_results("Included", "Include")
print_sorted_results("Subanalysis", "Subanalysis", show_reason=True)
print_sorted_results("Excluded", "Exclude", show_reason=True)

# --- Additional printed summary for excluded measurements breakdown ---
# Categories defined for exclusions
excluded_category_counts = {
    "ventilator malfunction": 0,
    "no respiratory activity in EMG channels": 0,
    "Motion artefacts diaphragm, Background noise parasternal": 0,
    "Motion artefacts parasternal, Background noise diaphragm": 0
}

for r in results:
    if r["Label"] != "Exclude":
        continue
    r_list = [x.strip().lower() for x in r["Reason"].split(";") if x.strip()]
    if any("ventilator malfunction" in s for s in r_list):
        category = "ventilator malfunction"
    elif (any("no respiratory activity diaphragm" in s for s in r_list) and 
          any("no respiratory activity parasternal" in s for s in r_list)):
        category = "no respiratory activity in EMG channels"
    elif (any("no respiratory activity parasternal" in s for s in r_list) and 
          any("motion artefacts diaphragm" in s for s in r_list)):
        category = "Motion artefacts diaphragm, Background noise parasastern(al)"  # slight variation; you may fix text
        category = "Motion artefacts diaphragm, Background noise parasternal"
    elif (any("no respiratory activity diaphragm" in s for s in r_list) and 
          any("motion artefacts parasternal" in s for s in r_list)):
        category = "Motion artefacts parasternal, Background noise diaphragm"
    else:
        category = "UNCLASSIFIED"
        print(f"Unclassified measurement: {r['Patient ID']} - {r['Time ID']} with reasons: {r_list}")
    if category in excluded_category_counts:
        excluded_category_counts[category] += 1
    else:
        excluded_category_counts[category] = 1

print("\n=== Excluded Measurements Breakdown by Category ===")
for cat, count in excluded_category_counts.items():
    percentage = (count / total) * 100 if total > 0 else 0
    print(f"{cat}: {count} measurements ({percentage:.2f}% of total measurements)")

# --- New: Additional printed summary for subanalysis measurements breakdown ---
subanalysis_records = [r for r in results if r["Label"] == "Subanalysis"]

# Initialize counters for subanalysis categories
subanalysis_counts = {
    "Background noise parasternal": 0,
    "Background noise diaphragm": 0,
    "Motion artefacts parasternal": 0,
    "Motion artefacts diaphragm": 0
}

for r in subanalysis_records:
    # Assume the subanalysis measurements should have one reason
    r_list = [x.strip().lower() for x in r["Reason"].split(";") if x.strip()]
    # Check for keywords in the reason string.
    if any("no respiratory activity parasternal" in s for s in r_list):
        subanalysis_counts["Background noise parasternal"] += 1
    if any("no respiratory activity diaphragm" in s for s in r_list):
        subanalysis_counts["Background noise diaphragm"] += 1
    if any("motion artefacts parasternal" in s for s in r_list):
        subanalysis_counts["Motion artefacts parasternal"] += 1
    if any("motion artefacts diaphragm" in s for s in r_list):
        subanalysis_counts["Motion artefacts diaphragm"] += 1

total_subanalysis = len(subanalysis_records)

print("\n=== Subanalysis Measurements Breakdown by Category ===")
for cat, count in subanalysis_counts.items():
    perc = (count / total_subanalysis * 100) if total_subanalysis > 0 else 0
    print(f"{cat}: {count} measurements ({perc:.2f}% of total measurements)")


In [ ]:
#CATEGORISATION SUMMARY 
import pandas as pd

###########################################
# PART 1: Extract durations from patient_data
###########################################

# Dictionary to store duration (in minutes) per measurement,
# keyed by (Patient ID, Time ID)
durations = {}

print("Extracting measurement durations:")

for pid, patient in patient_data.items():
    for tp, data in patient.items():
        try:
            # Retrieve the EMG channels (as in your visual inspection code)
            emg_ts = data['emg']
            # Use the time axis of the first EMG channel.
            t_data = emg_ts[0].t_data  # assume t_data is a list or array of time stamps (in seconds)
            if len(t_data) >= 2:
                t_min = float(t_data[0])
                t_max = float(t_data[-1])
                duration_seconds = t_max - t_min
                duration_minutes = duration_seconds / 60.0
            else:
                print(f"Patient {pid}, Time {tp}: Not enough time data samples.")
                duration_minutes = 0
            # Normalize keys: ensure patient ID is a string padded to 3 digits,
            # and time ID is stripped.
            key = (str(pid).strip().zfill(3), str(tp).strip())
            durations[key] = duration_minutes
            print(f"Patient {pid}, Time {tp}: Duration = {duration_seconds:.2f} seconds ({duration_minutes:.2f} minutes)")
        except Exception as e:
            print(f"Error retrieving duration for Patient {pid}, Time {tp}: {e}")
            key = (str(pid).strip().zfill(3), str(tp).strip())
            durations[key] = 0

###########################################
# PART 2: Read CSV and combine durations for summary
###########################################

# Read the measurement classification CSV file.
df = pd.read_csv("measurement_classification.csv")

# Normalize CSV keys so they match those used above:
# Convert Patient ID to string, strip whitespace, and zero-pad to length 3.
df['Patient ID'] = df['Patient ID'].astype(str).str.strip().apply(lambda x: x.zfill(3))
df['Time ID'] = df['Time ID'].astype(str).str.strip()

# Add a new column "Duration (min)" to the DataFrame using the durations dictionary.
def lookup_duration(row):
    key = (row['Patient ID'], row['Time ID'])
    return durations.get(key, 0)

df['Duration (min)'] = df.apply(lookup_duration, axis=1)

# --- Compute overall totals ---
total_measurements = len(df)
total_patients = df["Patient ID"].nunique()
total_duration = df['Duration (min)'].sum()

# --- Compute stats per label ---
labels = ["Include", "Subanalysis", "Exclude"]
summary_info = {}
for lab in labels:
    lab_df = df[df["Label"] == lab]
    num_meas = len(lab_df)
    unique_patients = lab_df["Patient ID"].nunique()
    percentage = (num_meas / total_measurements) * 100 if total_measurements > 0 else 0
    duration_minutes = lab_df['Duration (min)'].sum()
    summary_info[lab] = (num_meas, percentage, unique_patients, duration_minutes)

# --- Print the final summary ---
print(f"\nSummary for {total_measurements} measurements ({total_patients} patients): measured time: {total_duration:.0f} minutes")
print(f"  Included     : {summary_info['Include'][0]} ({summary_info['Include'][1]:.2f}%) across {summary_info['Include'][2]} patients: {summary_info['Include'][3]:.0f} minutes")
print(f"  Subanalysis  : {summary_info['Subanalysis'][0]} ({summary_info['Subanalysis'][1]:.2f}%) across {summary_info['Subanalysis'][2]} patients: {summary_info['Subanalysis'][3]:.0f} minutes")
print(f"  Exclude      : {summary_info['Exclude'][0]} ({summary_info['Exclude'][1]:.2f}%) across {summary_info['Exclude'][2]} patients: {summary_info['Exclude'][3]:.0f} minutes")


In [ ]:
#VISUEEL INSPECTION POST FILTERING
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import numpy as np

def scrollable_emg_plot(pid, tp, window_size=10, y_lims_emg=None, y_lims_vent=None):
    if pid not in patient_data or tp not in patient_data[pid]:
        print(f"❌ Invalid patient ID of time ID({pid}, {tp})")
        return

    emg_ts = patient_data[pid][tp]['emg']
    vent_ts = patient_data[pid][tp]['vent']

    # Neem tijd-as van eerste kanaal om globale tijd te bepalen
    t_data = emg_ts[0].t_data
    t_min = t_data[0]
    t_max = t_data[-1]
    t_slider_max = t_max - window_size

    # Slider setup
    @interact(t_start=FloatSlider(min=t_min, max=t_slider_max, step=1, value=t_min, description='Start (s)'))
    def update_plot(t_start):
        t_end = t_start + window_size

        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 8), sharex=True)
        axes_emg = axes[:2]
        axes_vent = axes[2]

        colors_clean = ['lightgray', 'lightgray']
        colors_env = ['tab:blue', 'tab:green']

        # Gefilterde EMG
        emg_ts.run(
            'plot_full',
            axes=axes_emg,
            signal_type='clean',
            colors=colors_clean,
            baseline_bool=False
        )

        # Envelope + baseline
        for ch_idx, ax in enumerate(axes_emg):
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_env, color=colors_env[ch_idx], linewidth=2.0, label='envelope')
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_baseline, color=colors_env[ch_idx], linestyle='--', linewidth=1.5, label='baseline')
            ax.legend(loc='upper right')

        axes_emg[0].set_title(f'EMG - patient {pid}, {tp}')
        axes_emg[-1].set_xlabel('Time (s)')

        # Ventilator
        vent_ts.run('plot_full', axes=[axes_vent])
        axes_vent.set_title(f'Ventilator - patient {pid}, {tp}')
        axes_vent.set_xlabel('Time (s)')

        # Zoom
        for ax in axes:
            ax.set_xlim([t_start, t_end])
        if y_lims_emg is not None:
            for ax in axes_emg:
                ax.set_ylim(y_lims_emg)
        if y_lims_vent is not None:
            axes_vent.set_ylim(y_lims_vent)

        plt.tight_layout()
        plt.show()

# Gebruik de interactieve plot
scrollable_emg_plot(
    pid='004',                 # patient id
    tp='t0',                   # time id
    window_size=20,            # time window
    y_lims_emg=[-5, 80],
    y_lims_vent=[-15, 20]
)



Peak detection EMG

In [ ]:
#EMG PEAK DETECTION
from resurfemg.postprocessing.event_detection import detect_emg_breaths
from resurfemg.postprocessing.event_detection import onoffpeak_baseline_crossing  # gebruik deze in plaats van slope_extrapolation

# Parameters
prominence_factor = 0.4
min_peak_width_s = 0.6  # bepaalt gevoeligheid

# Container voor resultaten
emg_peak_results = {}

for pid, patient in patient_data.items():
    print(f"\n Detecting EMG-peaks patient {pid}")
    emg_peak_results[pid] = {}

    for tp, data in patient.items():
        print(f"  Time {tp}:")
        emg_ts = data["emg"]
        fs = emg_ts.param["fs"]
        emg_peak_results[pid][tp] = {}

        for ch_idx, channel in enumerate(emg_ts.channels):
            y_env = channel.y_env
            y_baseline = channel.y_baseline

            # Detecteer pieken
            peak_idxs = detect_emg_breaths(
                emg_env=y_env,
                emg_baseline=y_baseline,
                threshold=0,
                prominence_factor=prominence_factor,
                min_peak_width_s=int(min_peak_width_s * fs)
            )

            # Detecteer onsets/offsets met baseline-crossing methode
            peak_start_idxs, peak_end_idxs, valid_starts, valid_ends, valid_peaks = onoffpeak_baseline_crossing(
                signal_env=y_env,
                baseline=y_baseline,
                peak_idxs=peak_idxs
            )

            # Opslaan geldige pieken
            channel_results = []
            for i, peak_idx in enumerate(peak_idxs):
                if valid_peaks[i]:
                    channel_results.append({
                        "peak_idx": int(peak_idx),
                        "onset_idx": int(peak_start_idxs[i]),
                        "offset_idx": int(peak_end_idxs[i])
                    })

            emg_peak_results[pid][tp][ch_idx] = channel_results
            print(f"    {len(channel_results)} valid peaks in channel {ch_idx}")

In [ ]:
#VISUAL INSPECTION EMG PEAKS
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def simple_emg_viewer_dual(pid='005', tp='t0', window_s=20):
    """
    Viewer voor beide EMG-kanalen en ventilator, met tijdslider en piekinspectie.
    
    Parameters
    ----------
    pid : str
        Patiënt-ID (zoals '005')
    tp : str
        Tijdstip (zoals 't0', 't1', 't3')
    window_s : int
        Lengte van het tijdsvenster in seconden
    """

    # Ophalen van data voor de juiste tijdsmeting
    emg_ts = patient_data[pid][tp]['emg']
    vent_ts = patient_data[pid][tp]['vent']
    fs = emg_ts.param['fs']

    # Tijdas (gelijk voor beide EMG-kanalen)
    t_env = emg_ts[0].t_data
    duration = t_env[-1]

    t_vent = vent_ts[0].t_data
    y_vent = vent_ts[0].y_raw
    y_vent_baseline = vent_ts[0].y_baseline

    # Slider
    t_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=duration - window_s,
        step=1,
        description='Start (s)',
        continuous_update=False
    )

    def plot_window(t_start):
        t_end = t_start + window_s

        fig, axs = plt.subplots(3, 1, figsize=(14, 8), sharex=True)

        colors = ['tab:green', 'tab:blue']
        labels = ['Parasternal', 'Diaphragm']

        # EMG-kanalen
        for ch_idx in [0, 1]:
            t_emg = emg_ts[ch_idx].t_data
            y_emg = emg_ts[ch_idx].y_env
            y_baseline = emg_ts[ch_idx].y_baseline
            peaks = emg_peak_results[pid][tp][ch_idx]

            ax = axs[ch_idx]
            ax.plot(t_emg, y_emg, color=colors[ch_idx], label=f"{labels[ch_idx]} envelope")
            ax.plot(t_emg, y_baseline, linestyle='--', color=colors[ch_idx], alpha=0.6, label="baseline")

            # Pieken + onset/offset
            for p in peaks:
                pt = t_emg[p['peak_idx']]
                onset = t_emg[p['onset_idx']]
                offset = t_emg[p['offset_idx']]
                if t_start <= pt <= t_end:
                    ax.plot(pt, y_emg[p['peak_idx']], 'o', color='red', label='piek' if p == peaks[0] else "")
                    ax.axvline(onset, linestyle='--', color='green', alpha=0.5, label='onset' if p == peaks[0] else "")
                    ax.axvline(offset, linestyle='--', color='orange', alpha=0.5, label='offset' if p == peaks[0] else "")

            ax.set_ylabel("EMG (µV)")
            ax.set_title(f'EMG - patient {pid}, {tp}')
            ax.set_xlim([t_start, t_end])
            ax.set_ylim([-5, 30])
            ax.legend(loc='upper right')

        # Ventilator
        ax3 = axs[2]
        ax3.plot(t_vent, y_vent, label='Paw', color='tab:blue')
        ax3.plot(t_vent, y_vent_baseline, linestyle='--', color='red', label='Baseline')
        ax3.set_xlim([t_start, t_end])
        ax3.set_ylim([-5, 20])
        ax3.set_ylabel("Paw (cmH₂O)")
        ax3.set_xlabel("Time (s)")
        ax3.set_title("Ventilator")
        ax3.legend()

        plt.tight_layout()
        plt.show()

    display(t_slider)
    widgets.interact(plot_window, t_start=t_slider)

simple_emg_viewer_dual(pid='005', tp='t0', window_s=20)

4. EMG peak assesment

In [ ]:
# EMG PEAK QUALITY ASSESSMENT
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.integrate import trapezoid
from resurfemg.postprocessing import quality_assessment as qa
from resurfemg.postprocessing import features

# Parameters
snr_threshold = 1.2
aub_threshold = 60  # percent
bellshape_threshold = 50  # percent

# Result container
results = []

# Loop over all patients and timepoints
for pid, patient in patient_data.items():
    for tp, data in patient.items():
        emg_ts = data["emg"]
        fs = emg_ts.param["fs"]
        t_data = emg_ts.channels[0].t_data  # time axis (the same for all channels)

        for ch_idx, channel in enumerate(emg_ts.channels):
            y_env = channel.y_env
            y_baseline = channel.y_baseline
            ch_label = emg_ts.labels[ch_idx] if ch_idx < len(emg_ts.labels) else f"channel_{ch_idx}"

            peaks = emg_peak_results[pid][tp][ch_idx]

            if len(peaks) == 0:
                print(f"⚠️ No peaks detected for patient {pid}, {tp}, channel {ch_idx}")
                continue

            peak_idxs = [p["peak_idx"] for p in peaks]
            onset_idxs = [p["onset_idx"] for p in peaks]
            offset_idxs = [p["offset_idx"] for p in peaks]

            # ➤ Calculate SNR
            snr_values = qa.snr_pseudo(y_env, peak_idxs, y_baseline, fs)

            # ➤ Area Under Baseline (AUB)
            valid_aub, perc_under, _ = qa.percentage_under_baseline(
                signal=y_env,
                fs=fs,
                peak_idxs=peak_idxs,
                start_idxs=onset_idxs,
                end_idxs=offset_idxs,
                baseline=y_baseline,
                aub_threshold=aub_threshold,
            )

            # ➤ Bell-curve error
            time_products = features.time_product(y_env, fs, onset_idxs, offset_idxs, y_baseline)
            valid_bell, _, perc_bell, _, _ = qa.evaluate_bell_curve_error(
                peak_idxs=peak_idxs,
                start_idxs=onset_idxs,
                end_idxs=offset_idxs,
                signal=y_env,
                fs=fs,
                time_products=time_products,
                bell_threshold=bellshape_threshold,
            )

            # ➤ For each peak, assign a label based on all three conditions.
            for i in range(len(peaks)):
                peak_time = t_data[peak_idxs[i]]
                onset_time = t_data[onset_idxs[i]]
                offset_time = t_data[offset_idxs[i]]

                label = (
                    "True peak" if (
                        snr_values[i] > snr_threshold and
                        perc_under[i] < aub_threshold and
                        perc_bell[i] < bellshape_threshold
                    ) else "False peak"
                )

                results.append({
                    "patient_id": pid,
                    "timepoint": tp,
                    "channel": ch_label,
                    "time_s": peak_time,
                    "onset_s": onset_time,
                    "offset_s": offset_time,
                    "snr": snr_values[i],
                    "perc_under_baseline": perc_under[i],
                    "perc_bellshape_error": perc_bell[i],
                    "label": label
                })

# Collect all results into one DataFrame and save to CSV.
results_df = pd.DataFrame(results)
save_path = Path("emg_peak_assessment_all_patients.csv")
results_df.to_csv(save_path, index=False)
print(f"\n✅ EMG-assessment stored in: {save_path}")

# ===============================
# FALSE PEAK BREAKDOWN SUMMARY PER CHANNEL
# ===============================
# We now want to see, separately for parasternal and diaphragm channels,
# the number and percentage (relative to that channel's total) of peaks
# failing each criterion or combination thereof.
file_path = 'emg_peak_assessment_all_patients.csv'
df = pd.read_csv(file_path)

for channel in ['parasternal', 'diaphragm']:
    ch_df = df[df['channel'] == channel]
    total_ch = len(ch_df)
    ch_false = ch_df[ch_df['label'] == 'False peak'].copy()
    total_false_ch = len(ch_false)
    
    # Create boolean columns indicating if the respective criterion was failed.
    ch_false['fails_snr'] = ch_false['snr'] <= snr_threshold
    ch_false['fails_aub'] = ch_false['perc_under_baseline'] >= aub_threshold
    ch_false['fails_bell'] = ch_false['perc_bellshape_error'] >= bellshape_threshold
    
    # Count breakdown (note: these counts are exclusive)
    count_all_three = ch_false[(ch_false['fails_snr']) & (ch_false['fails_aub']) & (ch_false['fails_bell'])].shape[0]
    count_snr_only = ch_false[(ch_false['fails_snr']) & (~ch_false['fails_aub']) & (~ch_false['fails_bell'])].shape[0]
    count_aub_only = ch_false[(ch_false['fails_aub']) & (~ch_false['fails_snr']) & (~ch_false['fails_bell'])].shape[0]
    count_bell_only = ch_false[(ch_false['fails_bell']) & (~ch_false['fails_snr']) & (~ch_false['fails_aub'])].shape[0]
    count_bell_aub_only = ch_false[(ch_false['fails_bell']) & (ch_false['fails_aub']) & (~ch_false['fails_snr'])].shape[0]
    count_bell_snr_only = ch_false[(ch_false['fails_bell']) & (ch_false['fails_snr']) & (~ch_false['fails_aub'])].shape[0]
    count_aub_snr_only = ch_false[(ch_false['fails_aub']) & (ch_false['fails_snr']) & (~ch_false['fails_bell'])].shape[0]
    
    print(f"\n=== False Peak Breakdown for {channel.capitalize()} ===")
    print(f"Total peaks: {total_ch}")
    print(f"False peaks (total): {total_false_ch} ({(total_false_ch/total_ch*100):.1f}%)")
    print(f"  - Failing all three (snr, AUB, bellshape): {count_all_three} ({(count_all_three/total_ch*100):.1f}%)")
    print(f"  - Failing SNR criterion only: {count_snr_only} ({(count_snr_only/total_ch*100):.1f}%)")
    print(f"  - Failing AUB criterion only: {count_aub_only} ({(count_aub_only/total_ch*100):.1f}%)")
    print(f"  - Failing bellshape criterion only: {count_bell_only} ({(count_bell_only/total_ch*100):.1f}%)")
    print(f"  - Failing both bellshape and AUB only: {count_bell_aub_only} ({(count_bell_aub_only/total_ch*100):.1f}%)")
    print(f"  - Failing both bellshape and SNR only: {count_bell_snr_only} ({(count_bell_snr_only/total_ch*100):.1f}%)")
    print(f"  - Failing both AUB and SNR only: {count_aub_snr_only} ({(count_aub_snr_only/total_ch*100):.1f}%)")

# ===============================
# CHANNEL-BASED EMG PEAK DETECTION SUMMARY
# ===============================
parasternal_peaks = df[df['channel'] == 'parasternal']
diaphragm_peaks = df[df['channel'] == 'diaphragm']

# Count total and true peaks per channel
total_parasternal = len(parasternal_peaks)
true_parasternal = (parasternal_peaks['label'] == 'True peak').sum()
percentage_true_parasternal = (true_parasternal / total_parasternal) * 100 if total_parasternal > 0 else 0

total_diaphragm = len(diaphragm_peaks)
true_diaphragm = (diaphragm_peaks['label'] == 'True peak').sum()
percentage_true_diaphragm = (true_diaphragm / total_diaphragm) * 100 if total_diaphragm > 0 else 0

print("\n=== EMG Peak Detection Summary ===")
print(f"Parasternal - Total peaks: {total_parasternal}, True peaks: {true_parasternal} ({percentage_true_parasternal:.1f}%)")
print(f"Diaphragm   - Total peaks: {total_diaphragm}, True peaks: {true_diaphragm} ({percentage_true_diaphragm:.1f}%)")


In [ ]:
import pandas as pd

# ---------------------------
# Part 1: Load classification file and select included measurements
# ---------------------------
msr_df = pd.read_csv("measurement_classification.csv")

# Create a key by concatenating Patient ID (padded to 3 digits) and Time ID
msr_df["key"] = msr_df["Patient ID"].astype(str).str.strip().str.zfill(3) + "_" + msr_df["Time ID"].astype(str).str.strip()

# Select only measurements labeled as "Include"
include_df = msr_df[msr_df["Label"] == "Include"]
included_keys = set(include_df["key"].unique())

# ---------------------------
# Part 2: Load EMG peak assessment results and filter to included measurements
# ---------------------------
peaks_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
peaks_df["key"] = peaks_df["patient_id"].astype(str).str.strip().str.zfill(3) + "_" + peaks_df["timepoint"].astype(str).str.strip()

# Filter to only include peaks from measurements labeled "Include"
filtered_peaks = peaks_df[peaks_df["key"].isin(included_keys)].copy()

# ---------------------------
# Part 3: Overall EMG Peak Detection Summary by Channel
# ---------------------------
summary = {}
for channel in ["parasternal", "diaphragm"]:
    ch_df = filtered_peaks[filtered_peaks["channel"].str.lower() == channel]
    total_peaks = len(ch_df)
    true_peaks = (ch_df["label"] == "True peak").sum()
    perc_true = (true_peaks / total_peaks * 100) if total_peaks > 0 else 0
    summary[channel] = (total_peaks, true_peaks, perc_true)

print("\n=== EMG Peak Detection Summary ===")
print(f"Parasternal - Total peaks: {summary['parasternal'][0]}, True peaks: {summary['parasternal'][1]} ({summary['parasternal'][2]:.1f}%)")
print(f"Diaphragm   - Total peaks: {summary['diaphragm'][0]}, True peaks: {summary['diaphragm'][1]} ({summary['diaphragm'][2]:.1f}%)")

# ---------------------------
# Part 4: False Peak Breakdown for Each Channel
# ---------------------------
# Thresholds
snr_threshold = 1.2
aub_threshold = 60      # in percent
bellshape_threshold = 50  # in percent

for channel in ["parasternal", "diaphragm"]:
    ch_df = filtered_peaks[filtered_peaks["channel"].str.lower() == channel]
    total_ch = len(ch_df)
    
    # Only consider false peaks.
    false_df = ch_df[ch_df["label"] == "False peak"].copy()
    total_false = len(false_df)
    
    # Create boolean columns indicating failure of each criterion.
    false_df['fails_snr'] = false_df['snr'] <= snr_threshold
    false_df['fails_aub'] = false_df['perc_under_baseline'] >= aub_threshold
    false_df['fails_bell'] = false_df['perc_bellshape_error'] >= bellshape_threshold
    
    # Count breakdown (counts calculated exclusively)
    count_all_three = false_df[(false_df['fails_snr']) & (false_df['fails_aub']) & (false_df['fails_bell'])].shape[0]
    count_snr_only = false_df[(false_df['fails_snr']) & (~false_df['fails_aub']) & (~false_df['fails_bell'])].shape[0]
    count_aub_only = false_df[(false_df['fails_aub']) & (~false_df['fails_snr']) & (~false_df['fails_bell'])].shape[0]
    count_bell_only = false_df[(false_df['fails_bell']) & (~false_df['fails_snr']) & (~false_df['fails_aub'])].shape[0]
    count_bell_aub_only = false_df[(false_df['fails_bell']) & (false_df['fails_aub']) & (~false_df['fails_snr'])].shape[0]
    count_bell_snr_only = false_df[(false_df['fails_bell']) & (false_df['fails_snr']) & (~false_df['fails_aub'])].shape[0]
    count_aub_snr_only = false_df[(false_df['fails_aub']) & (false_df['fails_snr']) & (~false_df['fails_bell'])].shape[0]
    
    print(f"\n=== False Peak Breakdown for {channel.capitalize()} ===")
    print(f"Total peaks: {total_ch}")
    print(f"False peaks (total): {total_false} ({(total_false/total_ch*100):.1f}%)")
    print(f"  - Failing all three (snr, AUB, bellshape): {count_all_three} ({(count_all_three/total_ch*100):.1f}%)")
    print(f"  - Failing SNR criterion only: {count_snr_only} ({(count_snr_only/total_ch*100):.1f}%)")
    print(f"  - Failing AUB criterion only: {count_aub_only} ({(count_aub_only/total_ch*100):.1f}%)")
    print(f"  - Failing bellshape criterion only: {count_bell_only} ({(count_bell_only/total_ch*100):.1f}%)")
    print(f"  - Failing both bellshape and AUB only: {count_bell_aub_only} ({(count_bell_aub_only/total_ch*100):.1f}%)")
    print(f"  - Failing both bellshape and SNR only: {count_bell_snr_only} ({(count_bell_snr_only/total_ch*100):.1f}%)")
    print(f"  - Failing both AUB and SNR only: {count_aub_snr_only} ({(count_aub_snr_only/total_ch*100):.1f}%)")



In [ ]:
#SUMMARY EMG PEAK ASSESMENT
import pandas as pd

# Inladen
df = pd.read_csv("emg_peak_assessment_all_patients.csv")

# Zorg dat patient_id een string is met voorloopnullen
df["patient_id"] = df["patient_id"].astype(str).str.zfill(3)

# Groepeer per patiënt, tijdstip én kanaal
summary = (
    df.groupby(["patient_id", "timepoint", "channel", "label"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Voeg totalen en percentages toe op patientniveau
summary["total_peaks"] = summary.get("True peak", 0) + summary.get("False peak", 0)
summary["true_peak_pct"] = (summary.get("True peak", 0) / summary["total_peaks"] * 100).round(1)

# Opslaan als .csv-bestand
summary.to_csv("emg_peak_summary.csv", index=False)
print(f"\n✅ EMG-summary stored in: emg_peak_summary.csv")

# Toon tabel per patient/cardinal level
print("\n🔍 EMG Peak Quality Assessment per Patient/Time ID/Channel:")
print(
    summary[
        [
            "patient_id",
            "timepoint",
            "channel",
            "True peak",
            "False peak",
            "total_peaks",
            "true_peak_pct",
        ]
    ].to_string(index=False)
)

# ---- Nieuwe toevoeging: Overzicht per kanaal ----

# Groepeer over kanaal en label om het totaal aantal peaks per channel te berekenen
channel_summary = (
    df.groupby(["channel", "label"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Voeg totalen en percentages toe op kanaalniveau
channel_summary["total_peaks"] = channel_summary.get("True peak", 0) + channel_summary.get("False peak", 0)
channel_summary["true_peak_pct"] = (
    channel_summary.get("True peak", 0) / channel_summary["total_peaks"] * 100
).round(1)

# Opslaan als .csv-bestand (optioneel)
channel_summary.to_csv("emg_peak_channel_summary.csv", index=False)
print(f"\n✅ EMG-channel-summary stored in: emg_peak_channel_summary.csv")

# Toon de samenvatting per kanaal
print("\n🔍Overall EMG Peak Quality Assessment per Channel:")
print(channel_summary.to_string(index=False))


Ventilator onset,end, trigger detection

In [ ]:
#VENTILATOR, START MV, END MV, TRIGGER DETECTION
import csv
from types import MethodType
import importlib
import resurfemg.preprocessing.airwaypressure as paw
importlib.reload(paw)

# --- Define the pipeline processing function for ventilator data ---
def pipeline_process_vent(self, threshold_small_peaks=None):
    """
    Process airway pressure data using the raw ventilator data.
    
    Assumes that the ventilator data is loaded as:
         vent_ts = patient_data[pid][tp]['vent']
    and that:
         t_vent = vent_ts[0].t_data
         y_vent = vent_ts[0].y_raw
         (fs will be set manually).
         
    Steps:
      1. Select the first time series from the ventilator data group.
      2. Manually set its sampling frequency (fs_vent = 100) and alias y_raw.
      3. Set self.channels to a list containing the chosen time series.
      4. Call the airwaypressure functions to process the data:
             - Find MV start (minima peaks)
             - Find MV end (maxima peaks)
             - Remove small peaks
             - Extract breathing trigger
             - Remove false MV starts
    """
    if threshold_small_peaks is None:
        threshold_small_peaks = 0.7

    # Use the ventilator data group (assumed list-like) and select the first timeseries.
    vent_channel = self[0]

    # Manually set the sampling frequency for ventilator data.
    fs_vent = 100
    vent_channel.fs = fs_vent  # Important: set fs so that airwaypressure functions work.
    
    # Set the raw ventilator data and sampling frequency on the overall object.
    self.y_raw = vent_channel.y_raw  
    self.fs = fs_vent

    # Create a dummy 'channels' attribute so that airwaypressure functions can access data via self.channels[0]
    self.channels = [vent_channel]

    # Process the airway pressure data:
    self.time_start_in_mv, self.peak_start_in_mv, self.idx_start_in_mv = paw.find_peaks_vent(
        self, 1.5, peak='minima'
    )
    self.time_end_in_mv, self.peak_end_in_mv, self.idx_end_in_mv = paw.find_peaks_vent(
        self, 1.5, peak='maxima'
    )
    self.time_end_in_mv, self.peak_end_in_mv, self.idx_end_in_mv = paw.remove_small_peaks(
        self, threshold_percentile=threshold_small_peaks
    )
    self.time_bt, self.bt, self.idx_bt = paw.extract_breathing_trigger(self)
    self.time_start_in_mv, self.peak_start_in_mv, self.idx_start_in_mv = paw.remove_false_mv_starts(self)

# --- Optionally, define a CSV save function for individual instances (if needed) ---
def save_pipeline_output_to_csv(self, output_filename="vent_pipeline_output.csv"):
    """
    Save the pipeline processed output for one measurement to a CSV file.
    The CSV file will include one row per computed parameter.
    """
    with open(output_filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Parameter", "Values"])
        writer.writerow(["time_start_in_mv", ", ".join(map(str, self.time_start_in_mv))])
        writer.writerow(["peak_start_in_mv", ", ".join(map(str, self.peak_start_in_mv))])
        writer.writerow(["idx_start_in_mv", ", ".join(map(str, self.idx_start_in_mv))])
        writer.writerow(["time_end_in_mv", ", ".join(map(str, self.time_end_in_mv))])
        writer.writerow(["peak_end_in_mv", ", ".join(map(str, self.peak_end_in_mv))])
        writer.writerow(["idx_end_in_mv", ", ".join(map(str, self.idx_end_in_mv))])
        writer.writerow(["time_bt", ", ".join(map(str, self.time_bt))])
        writer.writerow(["bt", ", ".join(map(str, self.bt))])
        writer.writerow(["idx_bt", ", ".join(map(str, self.idx_bt))])
    print(f"Individual pipeline output saved to {output_filename}")

# --- Aggregate processing for all patient and time IDs ---
# (Assuming patient_data is structured as:
#   patient_data[patient_id][time_id] is a dictionary with at least a key 'vent'
# )
results = []   # List to store processed results dictionaries.

for pid, patient in patient_data.items():
    for tp, data in patient.items():
        # Print a status message before processing
        print(f"calculating ventilator detection patient {pid}, {tp}...")
        
        # Get the ventilator data group for this measurement.
        vent_ts = data['vent']  # This is assumed list-like.
        
        # Attach the pipeline processing method to this instance.
        vent_ts.pipeline_process_vent = MethodType(pipeline_process_vent, vent_ts)
        # Run the pipeline processing method.
        vent_ts.pipeline_process_vent(threshold_small_peaks=0.7)
        
        # Print a ready message after processing.
        print(f"Ventilator detection ready patient {pid}, {tp}.")

        # Collect the processed output into a dictionary.
        rec = {
            "Patient ID": pid,
            "Time ID": tp,
            "time_start_in_mv": ", ".join(map(str, vent_ts.time_start_in_mv)),
            "peak_start_in_mv": ", ".join(map(str, vent_ts.peak_start_in_mv)),
            "idx_start_in_mv": ", ".join(map(str, vent_ts.idx_start_in_mv)),
            "time_end_in_mv": ", ".join(map(str, vent_ts.time_end_in_mv)),
            "peak_end_in_mv": ", ".join(map(str, vent_ts.peak_end_in_mv)),
            "idx_end_in_mv": ", ".join(map(str, vent_ts.idx_end_in_mv)),
            "time_bt": ", ".join(map(str, vent_ts.time_bt)),
            "bt": ", ".join(map(str, vent_ts.bt)),
            "idx_bt": ", ".join(map(str, vent_ts.idx_bt)),
        }
        results.append(rec)

# --- Write aggregated results to a CSV file ---
fieldnames = [
    "Patient ID", "Time ID",
    "time_start_in_mv", "peak_start_in_mv", "idx_start_in_mv",
    "time_end_in_mv", "peak_end_in_mv", "idx_end_in_mv",
    "time_bt", "bt", "idx_bt"
]

output_csv = "all_vent_pipeline_output.csv"
with open(output_csv, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print(f"All ventilator pipeline output saved to {output_csv}")


In [ ]:
# VISUAL INSPECTION EMG PEAKS VS VENTILATOR ONSET
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import numpy as np
import pandas as pd
from pathlib import Path

# Load the ventilator markers CSV
vent_markers_df = pd.read_csv("all_vent_pipeline_output.csv")

def parse_csv_array(val_str):
    """
    Parse a string representation of a numeric array.
    This version removes square brackets and extra quotes, then splits on commas.
    """
    if not val_str or not val_str.strip():
        return np.array([])
    # Remove any surrounding brackets and quotes:
    val_str = val_str.strip().strip("[]").replace("\"", "").replace("'", "")
    # Split on commas and convert to float:
    return np.array([float(x.strip()) for x in val_str.split(",") if x.strip() != ""])

# ----- Load EMG quality assessment CSV and build a dictionary -----
# Assumes the CSV file has the columns:
# "patient_id", "timepoint", "channel", "time_s", "onset_s", "offset_s", "snr", "perc_under_baseline", "perc_bellshape_error", "label"
emg_assessment_file = "emg_peak_assessment_all_patients.csv"
emg_assessment_df = pd.read_csv(emg_assessment_file)

# Build dictionary: emg_assessment[pid][tp][channel] = list of peak dicts.
# Normalize patient IDs (convert to int if possible to remove leading zeros) and convert channel names to lowercase.
emg_assessment = {}
for idx, row in emg_assessment_df.iterrows():
    pid_raw = str(row["patient_id"]).strip()
    if pid_raw.isdigit():
        pid_str = str(int(pid_raw))
    else:
        pid_str = pid_raw
    tp_str = str(row["timepoint"]).strip()
    ch = row["channel"].strip().lower()
    d = row.to_dict()
    emg_assessment.setdefault(pid_str, {}).setdefault(tp_str, {}).setdefault(ch, []).append(d)
# ---------------------------------------------------------------------

def scrollable_emg_plot(pid, tp, window_size=10, y_lims_emg=None, y_lims_vent=None):
    """
    Plots the EMG and ventilator signals interactively for patient `pid` and time `tp`.
    Overlays:
      - Ventilator markers from the CSV:
            • MV Start: (time_start_in_mv, peak_start_in_mv) as green triangles.
            • MV End:   (time_end_in_mv, peak_end_in_mv) as red inverted triangles.
            • Breathing Trigger: (time_bt, bt) as blue circles.
      - EMG peaks along with onset and offset markers:
            * Peak coordinates are taken from the global dictionary emg_peak_results[pid][tp][ch_idx].
            * The quality label is obtained from the quality assessment dictionary (emg_assessment).
              Peaks labeled as "True peak" are drawn with blue dots and those labeled as "False peak" with red dots.
              If no quality data is found, a default red dot is used.
    """
    # Normalize patient ID (remove leading zeros if numeric)
    pid_key = str(int(pid)) if str(pid).isdigit() else str(pid).strip()
    if pid not in patient_data or tp not in patient_data[pid]:
        print(f"❌ Invalid patient ID or time ID ({pid}, {tp})")
        return

    emg_ts = patient_data[pid][tp]['emg']
    vent_ts = patient_data[pid][tp]['vent']

    # Use the time-axis from the first EMG channel
    t_data = emg_ts[0].t_data
    t_min = t_data[0]
    t_max = t_data[-1]
    t_slider_max = t_max - window_size

    @interact(t_start=FloatSlider(min=t_min, max=t_slider_max, step=1,
                                   value=t_min, description='Start (s)'))
    def update_plot(t_start):
        t_end = t_start + window_size

        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 8), sharex=True)
        axes_emg = axes[:2]
        axes_vent = axes[2]

        colors_clean = ['lightgray', 'lightgray']
        colors_env = ['tab:blue', 'tab:green']

        # Plot the EMG channels using the run method (assumed to be defined for the timeseries objects)
        emg_ts.run('plot_full', axes=axes_emg, signal_type='clean',
                   colors=colors_clean, baseline_bool=False)
        for ch_idx, ax in enumerate(axes_emg):
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_env,
                    color=colors_env[ch_idx], linewidth=2.0, label='envelope')
            ax.plot(emg_ts[ch_idx].t_data, emg_ts[ch_idx].y_baseline,
                    color=colors_env[ch_idx], linestyle='--', linewidth=1.5, label='baseline')
            # ----- Overlay EMG peaks with onset/offset markers, color-coded by quality -----
            # Add dummy plots for legend: Blue for True peak, Red for False peak.
            ax.plot([], [], 'o', color='blue', markersize=8, label='True peak')
            ax.plot([], [], 'o', color='red', markersize=8, label='False peak')
            
            # Use global emg_peak_results for peak coordinates.
            if pid in emg_peak_results and tp in emg_peak_results[pid]:
                peaks = emg_peak_results[pid][tp][ch_idx]
                t_emg = emg_ts[ch_idx].t_data
                y_emg = emg_ts[ch_idx].y_env
                # Get channel label from the timeseries and normalize to lowercase.
                ch_label_orig = emg_ts.labels[ch_idx] if ch_idx < len(emg_ts.labels) else f"channel_{ch_idx}"
                ch_label = ch_label_orig.strip().lower()
                quality_data = None
                if pid_key in emg_assessment and tp in emg_assessment[pid_key]:
                    channel_map = {k.lower(): k for k in emg_assessment[pid_key][tp].keys()}
                    if ch_label in channel_map:
                        quality_data = emg_assessment[pid_key][tp][channel_map[ch_label]]
                for p in peaks:
                    pt = t_emg[p['peak_idx']]
                    onset = t_emg[p['onset_idx']]
                    offset = t_emg[p['offset_idx']]
                    if t_start <= pt <= t_end:
                        # Determine color based on quality label.
                        peak_color = 'red'  # default
                        if quality_data is not None:
                            match = None
                            for q in quality_data:
                                if abs(float(q["time_s"]) - pt) < 1e-3:
                                    match = q
                                    break
                            if match and match["label"].strip().lower() == "true peak":
                                peak_color = 'blue'
                        ax.plot(pt, y_emg[p['peak_idx']], 'o', color=peak_color, markersize=8)
                        ax.axvline(onset, linestyle='--', color='green', alpha=0.5)
                        ax.axvline(offset, linestyle='--', color='orange', alpha=0.5)
            else:
                print(f"⚠️ No peak data in global emg_peak_results for patient {pid}, {tp}, channel {ch_idx}")
            # -----------------------------------------------------------
            ax.legend(loc='upper right')
            ax.set_xlim([t_start, t_end])
            if y_lims_emg is not None:
                ax.set_ylim(y_lims_emg)
        axes_emg[0].set_title(f'EMG - patient {pid}, {tp}')
        axes_emg[-1].set_xlabel('Time (s)')

        # Plot the ventilator trace.
        vent_ts.run('plot_full', axes=[axes_vent])
        axes_vent.set_title(f'Ventilator - patient {pid}, {tp}')
        axes_vent.set_xlabel('Time (s)')
        if y_lims_vent is not None:
            axes_vent.set_ylim(y_lims_vent)

        # Overlay ventilator markers from the CSV.
        mask = ((vent_markers_df["Patient ID"].astype(str)
                             .str.strip().str.lstrip("0")) == str(pid).lstrip("0")) & \
               ((vent_markers_df["Time ID"].astype(str)
                             .str.strip()) == str(tp).strip())
        if mask.sum() > 0:
            row = vent_markers_df[mask].iloc[0]
            time_start_in_mv = parse_csv_array(row["time_start_in_mv"])
            peak_start_in_mv = parse_csv_array(row["peak_start_in_mv"])
            time_end_in_mv = parse_csv_array(row["time_end_in_mv"])
            peak_end_in_mv = parse_csv_array(row["peak_end_in_mv"])
            time_bt = parse_csv_array(row["time_bt"])
            bt = parse_csv_array(row["bt"])

            if time_start_in_mv.size and peak_start_in_mv.size:
                axes_vent.plot(time_start_in_mv, peak_start_in_mv, 'g^', markersize=10, label="MV Start")
            if time_end_in_mv.size and peak_end_in_mv.size:
                axes_vent.plot(time_end_in_mv, peak_end_in_mv, 'rv', markersize=10, label="MV End")
            if time_bt.size and bt.size:
                axes_vent.plot(time_bt, bt, 'bo', markersize=8, label="Breathing Trigger")
            axes_vent.legend(loc='upper right')
        else:
            print(f"No marker data found for patient {pid}, {tp}")

        plt.tight_layout()
        plt.show()

# Example usage:
scrollable_emg_plot(
    pid='002',       # example patient ID
    tp='t3',         # example time ID
    window_size=20,  # time window in seconds
    y_lims_emg=[-5, 80],
    y_lims_vent=[-5, 20]
)


In [ ]:
#ASSIGN VENTILATOR TRIGGER/AUTO TYPE
import pandas as pd
import numpy as np

def parse_csv_array(val):
    """
    Parse a value from the CSV that is expected to represent a numeric array.
    
    - If the value is missing (NaN), return an empty NumPy array.
    - If the value is not a string (e.g. a single numeric value), return a NumPy array with that item.
    - Otherwise, remove any extraneous brackets or quotes, split on commas,
      and return a NumPy array of floats.
    """
    # If the value is NaN, return an empty array.
    if pd.isnull(val):
        return np.array([])
    # If it's not a string, return a one-element array.
    if not isinstance(val, str):
        return np.array([val])
    # Remove any surrounding brackets and quotes.
    val_str = val.strip().strip("[]").replace("\"", "").replace("'", "")
    if not val_str:
        return np.array([])
    # Split on comma and convert each to float.
    return np.array([float(x.strip()) for x in val_str.split(",") if x.strip() != ""])

# Load the output CSV from the ventilator pipeline.
df = pd.read_csv("all_vent_pipeline_output.csv")

breath_records = []   # Will store one record per detected breath.
breath_idx_counter = 1

# Process each measurement (each row in the CSV).
for index, row in df.iterrows():
    pid = row["Patient ID"]
    tid = row["Time ID"]
    
    # Parse the marker arrays.
    ts = parse_csv_array(row["time_start_in_mv"])   # MV start times (x coordinate)
    ps = parse_csv_array(row["peak_start_in_mv"])     # MV start amplitudes
    te = parse_csv_array(row["time_end_in_mv"])       # MV end times (x coordinate)
    pe = parse_csv_array(row["peak_end_in_mv"])       # MV end amplitudes
    tb = parse_csv_array(row["time_bt"])              # Breathing trigger times (x coordinate)
    pb = parse_csv_array(row["bt"])                   # Breathing trigger amplitudes
    
    # Keep track of which MV start and breathing trigger indices have been used.
    used_ts = set()
    used_tb = set()
    
    # Process each detected MV end (te).
    for i, te_val in enumerate(te):
        window_start = te_val - 1.5
        
        # Find MV start candidates in the window [te - 1.5, te] that haven't been used.
        candidates_ts = []
        for j, ts_val in enumerate(ts):
            if j in used_ts:
                continue
            if window_start <= ts_val <= te_val:
                candidates_ts.append((ts_val, j))
        if len(candidates_ts) == 0:
            # No matching MV start found in this window; ignore this MV end.
            continue
        
        # Choose the MV start that is closest to the MV end (the maximum ts in the window).
        chosen_ts_val, chosen_ts_idx = max(candidates_ts, key=lambda x: x[0])
        
        # Check for breathing trigger candidates within the same window.
        candidates_tb = []
        for k, tb_val in enumerate(tb):
            if k in used_tb:
                continue
            if window_start <= tb_val <= te_val:
                candidates_tb.append((tb_val, k))
        if candidates_tb:
            # Select the candidate trigger closest to the MV end.
            chosen_tb_val, chosen_tb_idx = max(candidates_tb, key=lambda x: x[0])
            breath_type = "Triggered Breath"
        else:
            breath_type = "Auto Breath"
            chosen_tb_val = None
            chosen_tb_idx = None
        
        # Mark the chosen markers as used.
        used_ts.add(chosen_ts_idx)
        if chosen_tb_idx is not None:
            used_tb.add(chosen_tb_idx)
        
        # Create a record for this detected breath.
        rec = {
            "Patient ID": pid,
            "Time ID": tid,
            "Breath Index": breath_idx_counter,
            "Breath Type": breath_type,
            "time_start_in_mv": chosen_ts_val,
            "peak_start_in_mv": ps[chosen_ts_idx] if chosen_ts_idx < len(ps) else np.nan,
            "time_end_in_mv": te_val,
            "peak_end_in_mv": pe[i] if i < len(pe) else np.nan,
        }
        if breath_type == "Triggered Breath":
            rec["time_bt"] = chosen_tb_val
            rec["bt"] = pb[chosen_tb_idx] if chosen_tb_idx is not None and chosen_tb_idx < len(pb) else np.nan
        else:
            rec["time_bt"] = ""
            rec["bt"] = ""
            
        breath_records.append(rec)
        breath_idx_counter += 1

# Convert the detected breath records into a DataFrame and save as CSV.
breath_df = pd.DataFrame(breath_records)
breath_df.to_csv("vent_breaths_output.csv", index=False)
print("Breath detection CSV saved as vent_breaths_output.csv")

# ----------------------------------------------------------------
# Now, compute and print summary statistics.
total_breaths = len(breath_df)
total_triggered = (breath_df["Breath Type"] == "Triggered Breath").sum()
total_auto = (breath_df["Breath Type"] == "Auto Breath").sum()

if total_breaths > 0:
    percent_triggered = 100 * total_triggered / total_breaths
    percent_auto = 100 * total_auto / total_breaths
else:
    percent_triggered = 0
    percent_auto = 0

print("\nOverall Breath Detection Summary:")
print(f"  Total breaths: {total_breaths}")
print(f"  Triggered breaths: {total_triggered} ({percent_triggered:.2f}%)")
print(f"  Auto breaths: {total_auto} ({percent_auto:.2f}%)")

# Also, compute the numbers per patient/time measurement.
grouped = breath_df.groupby(["Patient ID", "Time ID"])
print("\nBreath Detection by Patient and Time ID:")
for (pid, tid), group in grouped:
    total = len(group)
    triggered = (group["Breath Type"] == "Triggered Breath").sum()
    auto = (group["Breath Type"] == "Auto Breath").sum()
    pct_triggered = 100 * triggered / total if total > 0 else 0
    pct_auto = 100 * auto / total if total > 0 else 0
    print(f"  Patient {pid}, Time {tid}:")
    print(f"    Total breaths: {total}")
    print(f"    Triggered: {triggered} ({pct_triggered:.2f}%)")
    print(f"    Auto: {auto} ({pct_auto:.2f}%)")




In [ ]:
import pandas as pd

# ---------------------------
# Part 1: Load measurement classification file and select included measurements
# ---------------------------
msr_df = pd.read_csv("measurement_classification.csv")

# Create a key by concatenating "Patient ID" and "Time ID"
msr_df["key"] = msr_df["Patient ID"].astype(str).str.strip() + "_" + msr_df["Time ID"].astype(str).str.strip()

# Select only measurements labeled as "Include"
include_df = msr_df[msr_df["Label"] == "Include"]
included_keys = set(include_df["key"].unique())

# ---------------------------
# Part 2: Load ventilator breath detection results
# ---------------------------
breath_df = pd.read_csv("vent_breaths_output.csv")

# Create the same key in the breath data
breath_df["key"] = breath_df["Patient ID"].astype(str).str.strip() + "_" + breath_df["Time ID"].astype(str).str.strip()

# ---------------------------
# Part 3: Filter for included measurements only
# ---------------------------
included_breaths = breath_df[breath_df["key"].isin(included_keys)].copy()

# ---------------------------
# Part 4: Compute overall summary for included measurements
# ---------------------------
total_breaths = len(included_breaths)
triggered_breaths = (included_breaths["Breath Type"] == "Triggered Breath").sum()
auto_breaths = (included_breaths["Breath Type"] == "Auto Breath").sum()

perc_triggered = (triggered_breaths / total_breaths * 100) if total_breaths > 0 else 0
perc_auto = (auto_breaths / total_breaths * 100) if total_breaths > 0 else 0

# ---------------------------
# Part 5: Print the overall summary
# ---------------------------
print("\n=== Overall Breath Detection Summary ===")
print(f"  Total breaths: {total_breaths}")
print(f"  Triggered breaths: {triggered_breaths} ({perc_triggered:.2f}%)")
print(f"  Auto breaths: {auto_breaths} ({perc_auto:.2f}%)")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# === Instellingen ===
assessment_file = "emg_peak_assessment_all_patients.csv"
pid = "005"
tp = "t0"
window = 0.5  # seconden
confidence = 0.95

# === CI-functie
def wilson_ci(x, n, confidence=0.95):
    if n == 0:
        return (0.0, 0.0)
    z = norm.ppf(1 - (1 - confidence) / 2)
    phat = x / n
    denom = 1 + (z**2 / n)
    centre = phat + (z**2 / (2 * n))
    margin = z * np.sqrt((phat * (1 - phat) + (z**2 / (4 * n))) / n)
    return (centre - margin) / denom * 100, (centre + margin) / denom * 100

# === Overlapfunctie
def compute_overlap(reference_times, comparison_times, window):
    count_overlap = sum(np.any(np.abs(comparison_times - t_ref) <= window) for t_ref in reference_times)
    percentage = (count_overlap / len(reference_times)) * 100 if len(reference_times) > 0 else 0
    return count_overlap, percentage

# === Data inladen ===
df = pd.read_csv(assessment_file)
df["patient_id"] = df["patient_id"].astype(str).str.zfill(3)

# Filter op patiënt, tijdstip en true peaks
df_sub = df[
    (df["patient_id"] == pid) &
    (df["timepoint"] == tp) &
    (df["label"] == "True peak")
]

# Check of kanaalnamen aanwezig zijn
channels = df_sub["channel"].unique()
if not {"parasternal", "diaphragm"}.issubset(set(channels)):
    print(f"⚠️ Not both channels are present for patient {pid}, {tp}: {channels}")
else:
    # Haal tijdstippen per kanaal
    t_para = df_sub[df_sub["channel"] == "parasternal"]["time_s"].to_numpy()
    t_diaphragm = df_sub[df_sub["channel"] == "diaphragm"]["time_s"].to_numpy()

    # Overlapberekening
    ov_para, perc_para = compute_overlap(t_para, t_diaphragm, window)
    ci_para = wilson_ci(ov_para, len(t_para))

    ov_dia, perc_dia = compute_overlap(t_diaphragm, t_para, window)
    ci_dia = wilson_ci(ov_dia, len(t_diaphragm))

    # Print output
    print(f"\nOverlap-analysis patient {pid}, {tp} (±{window}s):")
    print(f"Parasternal: {len(t_para)} peaks → {ov_para} overlap with diaphragm "
          f"({perc_para:.1f}%, 95% CI: {ci_para[0]:.1f}%–{ci_para[1]:.1f}%)")
    print(f"Diaphragm: {len(t_diaphragm)} peaks → {ov_dia} overlap with parasternal "
          f"({perc_dia:.1f}%, 95% CI: {ci_dia[0]:.1f}%–{ci_dia[1]:.1f}%)")

    # === Plot
    labels = ['Diaphragm', 'Parasternal']
    percentages = [perc_dia, perc_para]
    overlaps = [ov_dia, ov_para]
    counts = [len(t_diaphragm), len(t_para)]
    ci_lower = [p - ci[0] for p, ci in zip(percentages, [ci_dia, ci_para])]
    ci_upper = [ci[1] - p for p, ci in zip(percentages, [ci_dia, ci_para])]

    plt.figure(figsize=(6, 4))
    bars = plt.bar(labels, percentages, yerr=[ci_lower, ci_upper], capsize=5, color=["blue", "green"])
    plt.ylabel("Percentage overlapping peaks")
    plt.ylim(0, 110)
    plt.title(f"Overlap of True Peaks – Patient {pid}, {tp.upper()}")
    plt.grid(axis="y")

    for bar, total, n_overlap in zip(bars, counts, overlaps):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                 f"{n_overlap}/{total}", ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# === Instellingen ===
assessment_file = "emg_peak_assessment_all_patients.csv"
pid = "005"
tp = "t0"
window = 0.6  # seconden
confidence = 0.95

# === Wilson CI functie
def wilson_ci(x, n, confidence=0.95):
    if n == 0:
        return (0.0, 0.0)
    z = norm.ppf(1 - (1 - confidence) / 2)
    phat = x / n
    denom = 1 + (z**2 / n)
    centre = phat + (z**2 / (2 * n))
    margin = z * np.sqrt((phat * (1 - phat) + (z**2 / (4 * n))) / n)
    return (centre - margin) / denom * 100, (centre + margin) / denom * 100

# === Overlapfunctie
def compute_overlap(reference_times, comparison_times, window):
    count_overlap = sum(np.any(np.abs(comparison_times - t_ref) <= window) for t_ref in reference_times)
    percentage = (count_overlap / len(reference_times)) * 100 if len(reference_times) > 0 else 0
    return count_overlap, percentage

# === Data inladen ===
df = pd.read_csv(assessment_file)
df["patient_id"] = df["patient_id"].astype(str).str.zfill(3)

# Filter op patiënt en tijdstip
df_sub = df[(df["patient_id"] == pid) & (df["timepoint"] == tp)]

# Check of beide kanalen aanwezig zijn
channels = df_sub["channel"].unique()
if not {"parasternal", "diaphragm"}.issubset(set(channels)):
    print(f"⚠️ Niet beide kanalen aanwezig voor patiënt {pid}, {tp}: {channels}")
else:
    # Tijdstippen per kanaal en label
    t_para_true = df_sub[(df_sub["channel"] == "parasternal") & (df_sub["label"] == "True peak")]["time_s"].to_numpy()
    t_para_all = df_sub[df_sub["channel"] == "parasternal"]["time_s"].to_numpy()
    t_dia_true = df_sub[(df_sub["channel"] == "diaphragm") & (df_sub["label"] == "True peak")]["time_s"].to_numpy()
    t_dia_all = df_sub[df_sub["channel"] == "diaphragm"]["time_s"].to_numpy()

    # Overlap berekeningen
    ov_dia_true, perc_dia_true = compute_overlap(t_dia_true, t_para_true, window)
    ci_dia_true = wilson_ci(ov_dia_true, len(t_dia_true))

    ov_dia_all, perc_dia_all = compute_overlap(t_dia_true, t_para_all, window)
    ci_dia_all = wilson_ci(ov_dia_all, len(t_dia_true))

    ov_para_true, perc_para_true = compute_overlap(t_para_true, t_dia_true, window)
    ci_para_true = wilson_ci(ov_para_true, len(t_para_true))

    ov_para_all, perc_para_all = compute_overlap(t_para_true, t_dia_all, window)
    ci_para_all = wilson_ci(ov_para_all, len(t_para_true))

        # === Print output naar terminal
    print(f"\nOverlap-analysis patient {pid}, {tp} (±{window}s):")

    print("True peaks only:")
    print(f"  Parasternal: {len(t_para_true)} peaks → {ov_para_true} overlap with diaphragm "
          f"({perc_para_true:.1f}%, 95% CI: {ci_para_true[0]:.1f}%–{ci_para_true[1]:.1f}%)")
    print(f"  Diaphragm:   {len(t_dia_true)} peaks → {ov_dia_true} overlap with parasternal "
          f"({perc_dia_true:.1f}%, 95% CI: {ci_dia_true[0]:.1f}%–{ci_dia_true[1]:.1f}%)")

    print("\nCompared with all detected peaks:")
    print(f"  Parasternal: {len(t_para_true)} True peaks → {ov_para_all} overlap with all diaphragm peaks"
          f"({perc_para_all:.1f}%, 95% CI: {ci_para_all[0]:.1f}%–{ci_para_all[1]:.1f}%)")
    print(f"  Diaphragm:   {len(t_dia_true)} True peaks → {ov_dia_all} overlap with all parasternal peaks"
          f"({perc_dia_all:.1f}%, 95% CI: {ci_dia_all[0]:.1f}%–{ci_dia_all[1]:.1f}%)")

    # === Plot
    labels = ["Diaphragm", "Parasternal"]
    true_perc = [perc_dia_true, perc_para_true]
    all_perc = [perc_dia_all, perc_para_all]
    true_ci = [ci_dia_true, ci_para_true]
    all_ci = [ci_dia_all, ci_para_all]
    true_ov = [ov_dia_true, ov_para_true]
    all_ov = [ov_dia_all, ov_para_all]
    true_total = [len(t_dia_true), len(t_para_true)]

    x = np.arange(len(labels))  # [0, 1]
    width = 0.35

    fig, ax = plt.subplots(figsize=(8, 5))

    bars1 = ax.bar(x - width/2, true_perc, width,
                   yerr=[[p - c[0] for p, c in zip(true_perc, true_ci)],
                         [c[1] - p for p, c in zip(true_perc, true_ci)]],
                   capsize=5, color=['blue', 'blue'])

    bars2 = ax.bar(x + width/2, all_perc, width,
                   yerr=[[p - c[0] for p, c in zip(all_perc, all_ci)],
                         [c[1] - p for p, c in zip(all_perc, all_ci)]],
                   capsize=5, color=['lightblue', 'lightblue'])

    # Annotaties
    for i in range(len(x)):
        ax.text(x[i] - width/2, true_perc[i] + 2,
                f"{true_ov[i]}/{true_total[i]}", ha='center', va='bottom', fontsize=9)
        ax.text(x[i] + width/2, all_perc[i] + 2,
                f"{all_ov[i]}/{true_total[i]}", ha='center', va='bottom', fontsize=9, color='dimgray')

    # Aanduiding + legenda
    ax.set_ylabel("Percentage overlapping peaks")
    ax.set_ylim(0, 110)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_title(f"Overlap analyse – patiënt {pid}, tijdstip {tp.upper()}")
    legend_handles = [
        plt.Rectangle((0, 0), 1, 1, color='blue', label='True peaks'),
        plt.Rectangle((0, 0), 1, 1, color='lightblue', label='Compared with all peaks'),
        plt.Rectangle((0, 0), 1, 1, color='blue'),
        plt.Rectangle((0, 0), 1, 1, color='lightblue')
    ]
    ax.legend(handles=legend_handles[:2], loc="upper right")  # Alleen de blauwe legenda

    ax.grid(axis="y")
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interact, FloatSlider

# === Instellingen (pas hier eenvoudig aan) ===
pid = "002"
tp = "t24"
window = 0.5  # s: maximale verschil voor overlap
window_width = 120  # breedte van het venster in seconden

# === Data inladen ===
df = pd.read_csv("emg_peak_assessment_all_patients.csv")
df["patient_id"] = df["patient_id"].astype(str).str.zfill(3)

# Filter op patiënt, tijdstip en label
df_sub = df[
    (df["patient_id"] == pid) &
    (df["timepoint"] == tp) &
    (df["label"] == "True peak")
]

# Haal tijdstippen per kanaal
t_para = df_sub[df_sub["channel"] == "parasternal"]["time_s"].to_numpy()
t_dia = df_sub[df_sub["channel"] == "diaphragm"]["time_s"].to_numpy()

# Tijdsbereik
start_time = 0
end_time = max(t_para.max() if len(t_para) else 0, t_dia.max() if len(t_dia) else 0)

# === Plotfunctie ===
def plot_overlap_window(center_time):
    plt.figure(figsize=(12, 4))

    # Filter pieken binnen tijdsvenster
    mask_dia = (t_dia >= center_time - window_width/2) & (t_dia <= center_time + window_width/2)
    mask_para = (t_para >= center_time - window_width/2) & (t_para <= center_time + window_width/2)
    t_dia_window = t_dia[mask_dia]
    t_para_window = t_para[mask_para]

    # Plot pieken
    plt.scatter(t_dia_window, np.ones_like(t_dia_window), color="blue", label="Diaphragm peaks", marker='o')
    plt.scatter(t_para_window, np.zeros_like(t_para_window), color="green", label="Parasternal peaks", marker='x')

    # Lijnen bij overlap
    for t1 in t_dia_window:
        close = t_para_window[np.abs(t_para_window - t1) <= window]
        for t2 in close:
            plt.plot([t1, t2], [1, 0], color="red", alpha=0.5, linewidth=1)

    # Aankleding
    plt.yticks([0, 1], ["Parasternal", "Diaphragm"])
    plt.xlabel("Time (s)")
    plt.title(f"True peak overlap — patient {pid}, {tp} — ±{window}s")
    plt.xlim(center_time - window_width/2, center_time + window_width/2)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# === Interactieve slider aanroepen ===
if end_time > window_width:
    interact(plot_overlap_window,
             center_time=FloatSlider(min=start_time + window_width/2,
                                     max=end_time - window_width/2,
                                     step=1.0,
                                     value=(start_time + end_time)/2,
                                     description='Tijd (s)',
                                     continuous_update=False))
else:
    print(" Not enough data points for scrolling within this window")



Correlation EMG diafragma en EMG parasternal

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# === INVOER: pas deze waarden aan ===
pid = 5                # patiëntnummer (int)
tp = "t0"              # timepoint (bijv. "t0", "t1")
window = 0.5           # tijdsvenster in seconden
confidence = 0.95      # betrouwbaarheidsinterval

# === Bestanden inladen ===
emg_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
vent_df = pd.read_csv("ventilator_breaths_all_patients.csv")

# === Wilson CI functie ===
def wilson_ci(x, n, confidence=0.95):
    if n == 0:
        return (0.0, 0.0)
    z = norm.ppf(1 - (1 - confidence) / 2)
    phat = x / n
    denominator = 1 + (z**2 / n)
    centre = phat + (z**2 / (2 * n))
    margin = z * np.sqrt((phat * (1 - phat) + (z**2 / (4 * n))) / n)
    lower = (centre - margin) / denominator
    upper = (centre + margin) / denominator
    return lower * 100, upper * 100

# === Overlap-functie ===
def compute_overlap_percent(reference_peaks, comparison_peaks, window):
    count_overlap = 0
    for t_ref in reference_peaks:
        if np.any(np.abs(comparison_peaks - t_ref) <= window):
            count_overlap += 1
    percentage = (count_overlap / len(reference_peaks)) * 100 if len(reference_peaks) > 0 else 0
    return count_overlap, percentage

# === Analysefunctie ===
def analyse_emg_vs_vent_overlap(patient_id, timepoint, window=0.3, confidence=0.95):
    patient_id = int(patient_id)

    emg_sub = emg_df[(emg_df["patient_id"] == patient_id) & (emg_df["timepoint"] == timepoint)]
    vent_sub = vent_df[(vent_df["patient_id"] == patient_id) & (vent_df["timepoint"] == timepoint)]

    true_peaks = emg_sub[emg_sub["label"] == "True peak"]
    t_dia = true_peaks[true_peaks["channel"].str.contains("diaphragm", case=False)]["time_s"].to_numpy()
    t_int = true_peaks[true_peaks["channel"].str.contains("parasternal", case=False)]["time_s"].to_numpy()

    t_onsets = vent_sub["time_onset_s"].to_numpy()

    overlap_dia, perc_dia = compute_overlap_percent(t_dia, t_onsets, window)
    ci_dia = wilson_ci(overlap_dia, len(t_dia), confidence)

    overlap_int, perc_int = compute_overlap_percent(t_int, t_onsets, window)
    ci_int = wilson_ci(overlap_int, len(t_int), confidence)

    print(f" Overlapanalysis with ventilator-onsets (±{window}s window) patiënt {patient_id}, {timepoint}:")
    print(f"- Diaphragm: {len(t_dia)} true peaks → {overlap_dia} overlap "
          f"({perc_dia:.1f}%, 95% CI: {ci_dia[0]:.1f}% - {ci_dia[1]:.1f}%)")
    print(f"- Parasternal: {len(t_int)} true peaks → {overlap_int} overlap "
          f"({perc_int:.1f}%, 95% CI: {ci_int[0]:.1f}% - {ci_int[1]:.1f}%)")

    # === Visualisatie ===
    labels = ['Diaphragm', 'Parasternal']
    true_counts = [len(t_dia), len(t_int)]
    overlaps = [overlap_dia, overlap_int]
    percentages = [perc_dia, perc_int]
    ci_lower = [perc - ci[0] for perc, ci in zip(percentages, [ci_dia, ci_int])]
    ci_upper = [ci[1] - perc for perc, ci in zip(percentages, [ci_dia, ci_int])]

    plt.figure(figsize=(6, 4))
    bars = plt.bar(labels, percentages, yerr=[ci_lower, ci_upper], capsize=6, color=["blue", "green"])
    plt.ylabel("Percentage EMG peak overlap with ventilator onset")
    plt.ylim(0, 110)
    plt.title(f"Overlap EMG & Ventilator patient {patient_id}, {timepoint} (±{window}s)")
    plt.grid(axis="y")

    for bar, count, overlap in zip(bars, true_counts, overlaps):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3,
                 f"{overlap}/{count}", ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# === Uitvoeren ===
analyse_emg_vs_vent_overlap(pid, tp, window, confidence)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# === INVOER: pas deze waarden aan ===
pid = 5                # patiëntnummer (int)
tp = "t0"              # timepoint (bijv. "t0", "t1")
window = 0.5           # tijdsvenster in seconden
confidence = 0.95      # betrouwbaarheidsinterval

# === Bestanden inladen ===
emg_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
vent_df = pd.read_csv("vent_breaths_output.csv")

# === Wilson CI functie ===
def wilson_ci(x, n, confidence=0.95):
    if n == 0:
        return (0.0, 0.0)
    z = norm.ppf(1 - (1 - confidence) / 2)
    phat = x / n
    denominator = 1 + (z**2 / n)
    centre = phat + (z**2 / (2 * n))
    margin = z * np.sqrt((phat * (1 - phat) + (z**2 / (4 * n))) / n)
    lower = (centre - margin) / denominator
    upper = (centre + margin) / denominator
    return lower * 100, upper * 100

# === Aangepaste overlap-functie: alleen vóór ventilator-onsets ===
def compute_overlap_percent_prior_window(emg_peaks, vent_onsets, window):
    count_overlap = 0
    for t_emg in emg_peaks:
        if np.any((vent_onsets - window <= t_emg) & (t_emg < vent_onsets)):
            count_overlap += 1
    percentage = (count_overlap / len(emg_peaks)) * 100 if len(emg_peaks) > 0 else 0
    return count_overlap, percentage

# === Analysefunctie ===
def analyse_emg_vs_vent_overlap(patient_id, timepoint, window=0.3, confidence=0.95):
    patient_id = int(patient_id)

    emg_sub = emg_df[(emg_df["patient_id"] == patient_id) & (emg_df["timepoint"] == timepoint)]
    vent_sub = vent_df[(vent_df["patient_id"] == patient_id) & (vent_df["timepoint"] == timepoint)]

    true_peaks = emg_sub[emg_sub["label"] == "True peak"]
    t_dia = true_peaks[true_peaks["channel"].str.contains("diaphragm", case=False)]["time_s"].to_numpy()
    t_int = true_peaks[true_peaks["channel"].str.contains("parasternal", case=False)]["time_s"].to_numpy()

    t_onsets = vent_sub["time_end_in_mv"].to_numpy()

    overlap_dia, perc_dia = compute_overlap_percent_prior_window(t_dia, t_onsets, window)
    ci_dia = wilson_ci(overlap_dia, len(t_dia), confidence)

    overlap_int, perc_int = compute_overlap_percent_prior_window(t_int, t_onsets, window)
    ci_int = wilson_ci(overlap_int, len(t_int), confidence)

    print(f" Overlapanalyse met ventilator-onsets (in {window}s vóór onset) patiënt {patient_id}, {timepoint}:")
    print(f"- Diaphragm: {len(t_dia)} true peaks → {overlap_dia} overlap "
          f"({perc_dia:.1f}%, 95% CI: {ci_dia[0]:.1f}% - {ci_dia[1]:.1f}%)")
    print(f"- Parasternal: {len(t_int)} true peaks → {overlap_int} overlap "
          f"({perc_int:.1f}%, 95% CI: {ci_int[0]:.1f}% - {ci_int[1]:.1f}%)")

    # === Visualisatie ===
    labels = ['Diaphragm', 'Parasternal']
    true_counts = [len(t_dia), len(t_int)]
    overlaps = [overlap_dia, overlap_int]
    percentages = [perc_dia, perc_int]
    ci_lower = [perc - ci[0] for perc, ci in zip(percentages, [ci_dia, ci_int])]
    ci_upper = [ci[1] - perc for perc, ci in zip(percentages, [ci_dia, ci_int])]

    plt.figure(figsize=(6, 4))
    bars = plt.bar(labels, percentages, yerr=[ci_lower, ci_upper], capsize=6, color=["blue", "green"])
    plt.ylabel("Percentage EMG peak overlap with ventilator onset")
    plt.ylim(0, 110)
    plt.title(f"EMG peak ventilator onset overlap - patiënt {patient_id}, {timepoint}")
    plt.grid(axis="y")

    for bar, count, overlap in zip(bars, true_counts, overlaps):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3,
                 f"{overlap}/{count}", ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

# === Uitvoeren ===
analyse_emg_vs_vent_overlap(pid, tp, window, confidence)

In [ ]:
#ASYNCHRONY ANALYSIS ALL MEASUREMENTS
import pandas as pd
import numpy as np
import re

# === PARAMETERS ===
window_pre = 0.7       # window before ventilator onset (sec)
window_post = 0.3      # window after ventilator onset (sec)
window_overlap = 0.8   # window used for overlap between EMG channels

# === CUSTOM SORTING FUNCTIONS ===
def patient_sort_key(pid):
    try:
        return int(pid)
    except:
        return float('inf')

def timepoint_sort_key(tp):
    """
    Expects timepoint strings formatted like 't0', 't1', 't24', 't1v2', etc.
    Returns a tuple (time_value, version) where version defaults to 0 if not present.
    """
    m = re.match(r"t(\d+)(?:v(\d+))?", tp)
    if m:
        t_val = int(m.group(1))
        v_val = int(m.group(2)) if m.group(2) is not None else 0
        return (t_val, v_val)
    else:
        return (float('inf'), 0)

# === DATA LOADING ===
# Load the two CSV files.
emg_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
vent_df = pd.read_csv("vent_breaths_output.csv")  # from ventilator detection pipeline

# Ensure consistent string types.
emg_df["patient_id"] = emg_df["patient_id"].astype(str)
emg_df["timepoint"] = emg_df["timepoint"].astype(str)
vent_df["patient_id"] = vent_df["Patient ID"].astype(str)
vent_df["timepoint"] = vent_df["Time ID"].astype(str)

# Build a set of unique measurement pairs available in either dataset.
measurements = set()
for idx, row in vent_df.iterrows():
    measurements.add( (row["patient_id"], row["timepoint"]) )
for idx, row in emg_df.iterrows():
    measurements.add( (row["patient_id"], row["timepoint"]) )
# Now sort measurements by patient id (numerically) then by timepoint using our custom key.
measurements = sorted(list(measurements), key=lambda x: (patient_sort_key(x[0]), timepoint_sort_key(x[1])))

# === HELPER FUNCTIONS ===
def is_in_any_window(t_e, vent_array, window_pre, window_post):
    for t_v in vent_array:
        if (t_e >= t_v - window_pre) and (t_e <= t_v + window_post):
            return True
    return False

def compute_overlap_effective(emg_peaks, vent_onsets, window_pre, window_post):
    count = 0
    for t in emg_peaks:
        for v in vent_onsets:
            if (t >= v - window_pre) and (t <= v + window_post):
                count += 1
    return count

def compute_overlap(ref, comp, win):
    count = 0
    for t in ref:
        if np.any(np.abs(comp - t) <= win):
            count += 1
    return count

# === ANALYSIS ===
results = []  # to store one record per measurement

# Loop over each measurement in sorted order.
for pid, tp in measurements:
    # Subset data for this measurement from both CSVs.
    emg_sub = emg_df[(emg_df["patient_id"] == pid) & (emg_df["timepoint"] == tp)]
    vent_sub = vent_df[(vent_df["patient_id"] == pid) & (vent_df["timepoint"] == tp)]
    if emg_sub.empty or vent_sub.empty:
        continue

    # Use only "True peak" rows for EMG.
    true_emg = emg_sub[emg_sub["label"] == "True peak"]
    # Separate EMG channels by name.
    t_dia = true_emg[true_emg["channel"].str.contains("diaphragm", case=False)]["time_s"].to_numpy()
    t_int = true_emg[true_emg["channel"].str.contains("parasternal", case=False)]["time_s"].to_numpy()
    t_dia_sorted = np.sort(t_dia)
    t_int_sorted = np.sort(t_int)
    
    # Ventilator onsets.
    t_vent = vent_sub["time_end_in_mv"].to_numpy()
    
    # Initialize counts for triggered breaths.
    true_triggers = 0
    diaphragm_induced_triggers = 0
    parasternal_induced_triggers = 0
    auto_triggered = 0   # triggers not matched in originally triggered breaths
    
    # Initialize forced breath counts (for auto breaths).
    forced_breath = 0                  # Forced Breath: no EMG peak on either channel
    forced_breath_with_respiratory = 0 # with both diaphragm and parasternal activity
    forced_breath_with_diaphragm = 0     # only diaphragm activity
    forced_breath_with_parasternal = 0   # only parasternal activity

    # Used flags for candidate assignment in triggered analysis.
    used_dia = np.full(t_dia_sorted.shape, False)
    used_int = np.full(t_int_sorted.shape, False)
    
    # Iterate over each ventilator onset.
    for _, vent_row in vent_sub.iterrows():
        t_v = vent_row["time_end_in_mv"]
        breath_type_orig = vent_row["Breath Type"].strip()  # from vent_breaths_output.csv
        window_start = t_v - window_pre
        window_end = t_v + window_post

        if breath_type_orig != "Auto Breath":
            # For originally triggered breaths: assign EMG candidates
            cand_dia_idx = np.where((t_dia_sorted >= window_start) & (t_dia_sorted <= window_end) & (~used_dia))[0]
            cand_int_idx = np.where((t_int_sorted >= window_start) & (t_int_sorted <= window_end) & (~used_int))[0]
            if cand_dia_idx.size > 0 and cand_int_idx.size > 0:
                used_dia[cand_dia_idx[0]] = True
                used_int[cand_int_idx[0]] = True
                true_triggers += 1
            elif cand_dia_idx.size > 0 and cand_int_idx.size == 0:
                used_dia[cand_dia_idx[0]] = True
                diaphragm_induced_triggers += 1
            elif cand_int_idx.size > 0 and cand_dia_idx.size == 0:
                used_int[cand_int_idx[0]] = True
                parasternal_induced_triggers += 1
            else:
                auto_triggered += 1
        else:
            # For auto breaths: force the classification.
            count_dia = np.sum((t_dia_sorted >= window_start) & (t_dia_sorted <= window_end))
            count_int = np.sum((t_int_sorted >= window_start) & (t_int_sorted <= window_end))
            if count_dia == 0 and count_int == 0:
                forced_breath += 1
            elif count_dia > 0 and count_int > 0:
                forced_breath_with_respiratory += 1
            elif count_dia > 0 and count_int == 0:
                forced_breath_with_diaphragm += 1
            elif count_dia == 0 and count_int > 0:
                forced_breath_with_parasternal += 1

    total_triggered = int(true_triggers + diaphragm_induced_triggers + parasternal_induced_triggers + auto_triggered)
    total_forced = int(forced_breath + forced_breath_with_respiratory + forced_breath_with_diaphragm + forced_breath_with_parasternal)
    
    # --- WASTED EFFORTS ---
    wasted_dia = 0
    wasted_int = 0
    wasted_both = 0
    checked_times = set()
    emg_all = np.union1d(t_dia, t_int)
    for t_e in emg_all:
        if any(np.isclose(t_e, list(checked_times), atol=0.01)):
            continue
        if not is_in_any_window(t_e, t_vent, window_pre, window_post):
            found_dia = np.any(np.isclose(t_e, t_dia, atol=0.01))
            found_int = np.any(np.isclose(t_e, t_int, atol=0.01))
            if found_dia and not found_int:
                wasted_dia += 1
            elif found_int and not found_dia:
                wasted_int += 1
            elif found_dia and found_int:
                wasted_both += 1
        checked_times.add(t_e)
    
    # --- OVERLAPS ---
    overlap_dia = compute_overlap_effective(t_dia, t_vent, window_pre, window_post)
    overlap_int = compute_overlap_effective(t_int, t_vent, window_pre, window_post)
    overlap_dia_vs_int = compute_overlap(t_dia, t_int, window_overlap)
    overlap_int_vs_dia = compute_overlap(t_int, t_dia, window_overlap)
    
    # --- ASYNCHRONY INDEX ---
    # Denom includes ventilator onsets plus wasted efforts.
    asynchrony_index = (auto_triggered + wasted_both + wasted_dia + wasted_int) / (len(t_vent) + wasted_both + wasted_dia + wasted_int if len(t_vent) > 0 else 1)
    
    # Build the result record for this measurement.
    res = {
        "patient_id": pid,
        "timepoint": tp,
        "vent_onsets": len(t_vent),
        "n_diaphragm_true_peaks": len(t_dia),
        "n_parasternal_true_peaks": len(t_int),
        "true_triggers": true_triggers,
        "diaphragm_induced_triggers": diaphragm_induced_triggers,
        "parasternal_induced_triggers": parasternal_induced_triggers,
        "auto_triggers": auto_triggered,
        "forced_breath": forced_breath,
        "forced_breath_with_respiratory": forced_breath_with_respiratory,
        "forced_breath_with_diaphragm": forced_breath_with_diaphragm,
        "forced_breath_with_parasternal": forced_breath_with_parasternal,
        "wasted_effort_both": wasted_both,
        "wasted_effort_diaphragm": wasted_dia,
        "wasted_effort_parasternal": wasted_int,
        "overlap_diaphragm": overlap_dia,
        "overlap_parasternal": overlap_int,
        "dia_int_overlap": overlap_dia_vs_int,
        "int_dia_overlap": overlap_int_vs_dia,
        "asynchrony_index": asynchrony_index
    }
    results.append(res)

# Convert results to a DataFrame, then sort it by patient id and timepoint.
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(
    by=["patient_id", "timepoint"],
    key=lambda col: col.map(lambda x: patient_sort_key(x) if col.name == "patient_id" else timepoint_sort_key(x) if col.name == "timepoint" else x)
)
results_df.to_csv("asynchrony_analysis_updated.csv", index=False)
print("Per-measurement output saved as asynchrony_analysis_updated.csv")

# === PRINT SUMMARY PER MEASUREMENT IN DESIRED ORDER ===
for _, row in results_df.iterrows():
    forced_simple = int(row["forced_breath"])
    forced_resp = int(row["forced_breath_with_respiratory"])
    forced_dia = int(row["forced_breath_with_diaphragm"])
    forced_int = int(row["forced_breath_with_parasternal"])
    total_forced = forced_simple + forced_resp + forced_dia + forced_int
    total_triggered = int(row["true_triggers"] + row["diaphragm_induced_triggers"] + row["parasternal_induced_triggers"] + row["auto_triggers"])
    
    print(f"\nAnalysis for patient {row['patient_id']}, timepoint {row['timepoint']}:")
    print(f"- Number of ventilator-onsets: {row['vent_onsets']}")
    print(f"- Number of diaphragm true peaks: {row['n_diaphragm_true_peaks']}")
    print(f"- Number of parasternal true peaks: {row['n_parasternal_true_peaks']}")
    print(f"- Forced breaths (total): {total_forced} ({100*total_forced/row['vent_onsets']:.2f}%)")
    print(f"    True Forced Breath: {forced_simple} ")
    print(f"    Forced Breath with Total Respiratory activity: {forced_resp} ")
    print(f"    Forced Breath with Diaphragm activity: {forced_dia} ")
    print(f"    Forced Breath with Parasternal activity: {forced_int} ")
    print(f"- Triggered breaths (total): {total_triggered} ({100*total_triggered/row['vent_onsets']:.2f}%)")
    print(f"    Auto-Triggers {row['auto_triggers']}")
    print(f"    True Triggers: {row['true_triggers']}")
    print(f"    Parasternal Induced Triggers: {row['parasternal_induced_triggers']}")
    print(f"    Diaphragm Induced Triggers: {row['diaphragm_induced_triggers']}")
    print(f"- Complete Wasted Effort (both EMG, no vent): {row['wasted_effort_both']}")
    print(f"- Wasted Effort Diaphragm: {row['wasted_effort_diaphragm']}")
    print(f"- Wasted Effort Parasternal: {row['wasted_effort_parasternal']}")
    print(f"- Overlap diaphragm ↔ vent: {row['overlap_diaphragm']}/{row['n_diaphragm_true_peaks']}")
    print(f"- Overlap parasternal ↔ vent: {row['overlap_parasternal']}/{row['n_parasternal_true_peaks']}")
    print(f"- Overlap diaphragm ↔ parasternal: {row['dia_int_overlap']}/{row['n_diaphragm_true_peaks']}")
    print(f"- Overlap parasternal ↔ diaphragm: {row['int_dia_overlap']}/{row['n_parasternal_true_peaks']}")
    print(f"- Asynchrony Index: {row['asynchrony_index']:.3f}")


In [ ]:
import pandas as pd

# ---- Part 1: Load classification & select included measurements ----
msr_df = pd.read_csv("measurement_classification.csv")
# Construct a key as "PatientID_TimeID"
msr_df["key"] = msr_df["Patient ID"].astype(str).str.strip() + "_" + msr_df["Time ID"].astype(str).str.strip()
# Keep only measurements labeled as "Include"
included_keys = set(msr_df[msr_df["Label"] == "Include"]["key"].unique())

# ---- Part 2: Load EMG peak assessment data and filter for true peaks from included measurements ----
peaks_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
# Construct same key from patient_id and timepoint
peaks_df["key"] = peaks_df["patient_id"].astype(str).str.strip() + "_" + peaks_df["timepoint"].astype(str).str.strip()
# Filter: only true peaks and only from included measurements
true_peaks = peaks_df[(peaks_df["label"] == "True peak") & (peaks_df["key"].isin(included_keys))].copy()

# ---- Part 3: Compute overlap counts using a 0.8-second window for each measurement ----
window = 0.8  # seconds

# We'll accumulate overall counts.
total_dia = 0
total_para = 0
overlap_dia = 0
overlap_para = 0

# Group by measurement key.
grouped = true_peaks.groupby("key")
for key, group in grouped:
    # Extract peak times for each channel; assume channel names are case-insensitive.
    dia_times = group[group["channel"].str.lower() == "diaphragm"]["time_s"].values
    para_times = group[group["channel"].str.lower() == "parasternal"]["time_s"].values

    n_dia = len(dia_times)
    n_para = len(para_times)
    total_dia += n_dia
    total_para += n_para

    # For each diaphragm peak, count if there's any parasternal peak within the window.
    cnt_dia = sum(any(abs(d - p) <= window for p in para_times) for d in dia_times)
    # For each parasternal peak, count if there's any diaphragm peak within the window.
    cnt_para = sum(any(abs(p - d) <= window for d in dia_times) for p in para_times)

    overlap_dia += cnt_dia
    overlap_para += cnt_para

# To force symmetry, take the average of the two overlap counts.
common_overlap = (overlap_dia + overlap_para) / 2

# Compute overlap percentages per channel.
perc_dia = (common_overlap / total_dia * 100) if total_dia > 0 else 0
perc_para = (common_overlap / total_para * 100) if total_para > 0 else 0

# ---- Part 4: Print results in the requested format ----
print("Diaphragm with Parasternal:")
print(f"  Total diaphragm true peaks: {total_dia}")
print(f"  Overlap (diaphragm with parasternal): {int(common_overlap)} ({perc_dia:.1f}%)\n")

print("Parasternal with Diaphragm:")
print(f"  Total parasternal true peaks: {total_para}")
print(f"  Overlap (parasternal with diaphragm): {int(common_overlap)} ({perc_para:.1f}%)")


In [ ]:
#ASYNCHRONY SUMMARY ALL MEASUREMENTS
import pandas as pd

# Load the asynchrony analysis CSV file.
df = pd.read_csv("asynchrony_analysis_updated.csv")

# List of fields to sum.
sum_fields = [
    "vent_onsets",
    "n_diaphragm_true_peaks",
    "n_parasternal_true_peaks",
    "true_triggers",
    "diaphragm_induced_triggers",
    "parasternal_induced_triggers",
    "auto_triggers",
    "forced_breath",
    "forced_breath_with_respiratory",
    "forced_breath_with_diaphragm",
    "forced_breath_with_parasternal",
    "wasted_effort_both",
    "wasted_effort_diaphragm",
    "wasted_effort_parasternal",
    "overlap_diaphragm",
    "overlap_parasternal",
    "dia_int_overlap",
    "int_dia_overlap"
]

# Compute totals for each field.
totals = {field: df[field].sum() for field in sum_fields}

# Get total ventilator onsets.
vent_onsets = totals["vent_onsets"]

# Total wasted efforts is the sum of the different wasted effort fields.
total_wasted = totals["wasted_effort_both"] + totals["wasted_effort_diaphragm"] + totals["wasted_effort_parasternal"]

# Total breaths is defined as ventilator onsets plus wasted efforts.
total_breaths = vent_onsets + total_wasted

# Total triggered breaths.
total_triggered = int(
    totals["true_triggers"] +
    totals["diaphragm_induced_triggers"] +
    totals["parasternal_induced_triggers"] +
    totals["auto_triggers"]
)

# Total forced breaths.
total_forced = int(
    totals["forced_breath"] +
    totals["forced_breath_with_respiratory"] +
    totals["forced_breath_with_diaphragm"] +
    totals["forced_breath_with_parasternal"]
)

# Calculate percentages relative to total breaths.
pct_triggered_total = 100 * total_triggered / total_breaths if total_breaths > 0 else 0
pct_forced_total    = 100 * total_forced / total_breaths if total_breaths > 0 else 0
pct_wasted_total    = 100 * total_wasted / total_breaths if total_breaths > 0 else 0

# Also, calculate percentages relative to ventilator onsets.
pct_triggered_vent = 100 * total_triggered / vent_onsets if vent_onsets > 0 else 0
pct_forced_vent    = 100 * total_forced / vent_onsets if vent_onsets > 0 else 0
pct_wasted_vent    = 100 * total_wasted / vent_onsets if vent_onsets > 0 else 0

# Compute the mean asynchrony index.
mean_asynchrony_index = df["asynchrony_index"].mean()

# --- PRINT THE OVERALL SUMMARY ---
print("Overall Summary of Asynchrony Analysis:")
print("----------------------------------------")
print(f"Total breaths (ventilator onsets + wasted efforts): {total_breaths}")
print(f"Number of ventilator-onsets: {vent_onsets}")
print(f"Number of diaphragm true peaks: {totals['n_diaphragm_true_peaks']}")
print(f"Number of parasternal true peaks: {totals['n_parasternal_true_peaks']}")
print(f" -Total Triggered Breaths: {total_triggered} ({pct_triggered_total:.2f}% of total breaths, {pct_triggered_vent:.2f}% of ventilator onsets)")
print(f"     True triggers: {totals['true_triggers']}")
print(f"     Auto-Triggers: {totals['auto_triggers']}")
print(f"     Diaphragm induced triggers: {totals['diaphragm_induced_triggers']}")
print(f"     Parasternal induced triggers: {totals['parasternal_induced_triggers']}")
print(f" -Total Forced Breaths: {total_forced} ({pct_forced_total:.2f}% of total breaths, {pct_forced_vent:.2f}% of ventilator onsets)")
print(f"     True Forced Breath: {totals['forced_breath']}")
print(f"     Forced Breath with Respiratory activity: {totals['forced_breath_with_respiratory']}")
print(f"     Forced Breath with Diaphragm activity: {totals['forced_breath_with_diaphragm']}")
print(f"     Forced Breath with Parasternal activity: {totals['forced_breath_with_parasternal']}")
print(f" -Total Wasted Efforts: {total_wasted} ({pct_wasted_total:.2f}% of total breaths)")
print(f"     Complete Wasted Effort (both EMG, no vent): {totals['wasted_effort_both']}")
print(f"     Wasted Effort Diaphragm: {totals['wasted_effort_diaphragm']}")
print(f"     Wasted Effort Parasternal: {totals['wasted_effort_parasternal']}")

print(f"Overlap diaphragm ↔ vent: {totals['overlap_diaphragm']}/{totals['n_diaphragm_true_peaks']}")
print(f"Overlap parasternal ↔ vent: {totals['overlap_parasternal']}/{totals['n_parasternal_true_peaks']}")
print(f"Overlap diaphragm ↔ parasternal: {totals['dia_int_overlap']}/{totals['n_diaphragm_true_peaks']}")
print(f"Overlap parasternal ↔ diaphragm: {totals['int_dia_overlap']}/{totals['n_parasternal_true_peaks']}")
print(f"Mean asynchrony index: {mean_asynchrony_index:.3f}")



In [ ]:
#ASYNCHRONY ANALYSIS INCLUDED MEASUREMENTS
import pandas as pd

# --- LOAD THE QUALITY CHECK FILE ---
inc_df = pd.read_csv("measurement_classification.csv")
# We'll assume that the included_measurements.csv file has columns "Patient ID", "Time ID", "Label", "Reasons"
# Filter to keep only included measurements:
#inc_df = inc_df[inc_df["Label"].str.strip().str.lower() == "include"]

# Filter to keep only subanalysis measurements:
inc_df = inc_df[inc_df["Label"].str.strip().str.lower() == "subanalysis"] #!!!aanpassen voor subanalysis

# Build a set of keys (patient, timepoint) from the quality-check file.
# Convert the values to strings and then strip extra whitespace.
included_keys = set()
for idx, row in inc_df.iterrows():
    pid = str(row["Patient ID"]).strip()
    tp = str(row["Time ID"]).strip()
    included_keys.add((pid, tp))

# --- LOAD THE ASYNCHRONY ANALYSIS CSV ---
df = pd.read_csv("asynchrony_analysis_updated.csv")
# Ensure consistency by stripping whitespace.
df["patient_id"] = df["patient_id"].astype(str).str.strip()
df["timepoint"] = df["timepoint"].astype(str).str.strip()

# Filter to include only those rows whose (patient_id, timepoint) key is in included_keys.
df_inc = df[df.apply(lambda row: (row["patient_id"], row["timepoint"]) in included_keys, axis=1)]

# --- PRINT HOW MANY PATIENTS and MEASUREMENTS ARE INCLUDED ---
unique_patients = df_inc["patient_id"].nunique()
total_measurements = len(df_inc)
print(f"Summary is performed on {unique_patients} patients and {total_measurements} measurements.")

# --- SUMMARIZATION SCRIPT (for included measurements) ---

# Define the list of fields to sum.
sum_fields = [
    "vent_onsets",
    "n_diaphragm_true_peaks",
    "n_parasternal_true_peaks",
    "true_triggers",
    "diaphragm_induced_triggers",
    "parasternal_induced_triggers",
    "auto_triggers",
    "forced_breath",
    "forced_breath_with_respiratory",
    "forced_breath_with_diaphragm",
    "forced_breath_with_parasternal",
    "wasted_effort_both",
    "wasted_effort_diaphragm",
    "wasted_effort_parasternal",
    "overlap_diaphragm",
    "overlap_parasternal",
    "dia_int_overlap",
    "int_dia_overlap"
]

# Compute totals for each field over the included measurements.
totals = {field: df_inc[field].sum() for field in sum_fields}

# Get total ventilator onsets.
vent_onsets = totals["vent_onsets"]

# Total wasted efforts is the sum of wasted_effort_both, wasted_effort_diaphragm, and wasted_effort_parasternal.
total_wasted = totals["wasted_effort_both"] + totals["wasted_effort_diaphragm"] + totals["wasted_effort_parasternal"]

# Total breaths is defined as ventilator onsets plus wasted efforts.
total_breaths = vent_onsets + total_wasted

# Total triggered breaths.
total_triggered = int(
    totals["true_triggers"] +
    totals["diaphragm_induced_triggers"] +
    totals["parasternal_induced_triggers"] +
    totals["auto_triggers"]
)

# Total forced breaths.
total_forced = int(
    totals["forced_breath"] +
    totals["forced_breath_with_respiratory"] +
    totals["forced_breath_with_diaphragm"] +
    totals["forced_breath_with_parasternal"]
)

# Calculate percentages relative to total breaths.
pct_triggered_total = 100 * total_triggered / total_breaths if total_breaths > 0 else 0
pct_forced_total    = 100 * total_forced / total_breaths if total_breaths > 0 else 0
pct_wasted_total    = 100 * total_wasted / total_breaths if total_breaths > 0 else 0

# Also, calculate percentages relative to ventilator onsets.
pct_triggered_vent = 100 * total_triggered / vent_onsets if vent_onsets > 0 else 0
pct_forced_vent    = 100 * total_forced / vent_onsets if vent_onsets > 0 else 0
pct_wasted_vent    = 100 * total_wasted / vent_onsets if vent_onsets > 0 else 0

# Compute the mean asynchrony index.
mean_asynchrony_index = df_inc["asynchrony_index"].mean()

# --- PRINT THE OVERALL SUMMARY ---
print("\nOverall Summary of Asynchrony Analysis (Included Measurements):")
print("----------------------------------------------------------------")
print(f"Total breaths (ventilator onsets + wasted efforts): {total_breaths}")
print(f"Number of ventilator-onsets: {vent_onsets}")
print(f"Number of diaphragm true peaks: {totals['n_diaphragm_true_peaks']}")
print(f"Number of parasternal true peaks: {totals['n_parasternal_true_peaks']}")
print(f"  - Total Triggered Breaths: {total_triggered} ({pct_triggered_total:.2f}% of total breaths, {pct_triggered_vent:.2f}% of ventilator onsets)")
print(f"      True triggers: {totals['true_triggers']}")
print(f"      Auto-Triggers: {totals['auto_triggers']}")
print(f"      Diaphragm induced triggers: {totals['diaphragm_induced_triggers']}")
print(f"      Parasternal induced triggers: {totals['parasternal_induced_triggers']}")
print(f"  - Total Forced Breaths: {total_forced} ({pct_forced_total:.2f}% of total breaths, {pct_forced_vent:.2f}% of ventilator onsets)")
print(f"      True Forced Breath: {totals['forced_breath']}")
print(f"      Forced Breath with Parasternal and Diaphragm activity: {totals['forced_breath_with_respiratory']}")
print(f"      Forced Breath with Diaphragm activity: {totals['forced_breath_with_diaphragm']}")
print(f"      Forced Breath with Parasternal activity: {totals['forced_breath_with_parasternal']}")
print(f"  - Total Wasted Efforts: {total_wasted} ({pct_wasted_total:.2f}% of total breaths)")
print(f"      Complete Wasted Effort (both EMG, no vent): {totals['wasted_effort_both']}")
print(f"      Wasted Effort Diaphragm: {totals['wasted_effort_diaphragm']}")
print(f"      Wasted Effort Parasternal: {totals['wasted_effort_parasternal']}")
print(f"Overlap diaphragm ↔ ventilator: {totals['overlap_diaphragm']}/{totals['n_diaphragm_true_peaks']}")
print(f"Overlap parasternal ↔ ventilator: {totals['overlap_parasternal']}/{totals['n_parasternal_true_peaks']}")
print(f"Overlap diaphragm ↔ parasternal: {totals['dia_int_overlap']}/{totals['n_diaphragm_true_peaks']}")
print(f"Overlap parasternal ↔ diaphragm: {totals['int_dia_overlap']}/{totals['n_parasternal_true_peaks']}")
print(f"Mean asynchrony index: {mean_asynchrony_index:.3f}")



In [ ]:
#ASYNCHRONY ANALYSIS MEASUREMENTS LABELED SUBANALYSIS
import pandas as pd

# ---------------------------
# Part 1: Load measurement classification and filter for Subanalysis measurements
# ---------------------------
class_df = pd.read_csv("measurement_classification.csv")
# Keep only measurements with Label equal to "Subanalysis" (case-insensitive)
class_df = class_df[class_df["Label"].str.strip().str.lower() == "subanalysis"]

# Create a key for each measurement as "PatientID_TimeID"
class_df["key"] = class_df["Patient ID"].astype(str).str.strip() + "_" + class_df["Time ID"].astype(str).str.strip()

# Determine the usable lead from the Reason.
# If Reason contains "no respiratory activity parasternal" or "motion artefacts parasternal" => use diaphragm.
# If Reason contains "no respiratory activity diaphragm" or "motion artefacts diaphragm" => use parasternal.
def determine_usable_lead(reason):
    r = reason.lower()
    if "no respiratory activity parasternal" in r or "motion artefacts parasternal" in r:
        return "diaphragm"
    elif "no respiratory activity diaphragm" in r or "motion artefacts diaphragm" in r:
        return "parasternal"
    else:
        return None

class_df["usable_lead"] = class_df["Reason"].apply(determine_usable_lead)

# Build the set of keys from subanalysis measurements.
subanalysis_keys = set(class_df["key"].unique())

# ---------------------------
# Part 2: Load asynchrony analysis data and filter to subanalysis measurements
# ---------------------------
ana_df = pd.read_csv("asynchrony_analysis_updated.csv")
# Create the same key from the asynchrony analysis data.
ana_df["key"] = ana_df["patient_id"].astype(str).str.strip() + "_" + ana_df["timepoint"].astype(str).str.strip()
# Filter to only measurements in subanalysis.
subana_df = ana_df[ana_df["key"].isin(subanalysis_keys)].copy()

# Merge the usable lead information from classification into the asynchrony analysis data.
merged_df = pd.merge(subana_df, class_df[["key", "usable_lead"]], on="key", how="left")

# ---------------------------
# Part 3: Split into two groups based on usable lead
# ---------------------------
# Group A: If usable_lead is "diaphragm", then use diaphragm metrics.
group_diaphragm = merged_df[merged_df["usable_lead"] == "diaphragm"]

# Group B: If usable_lead is "parasternal", then use parasternal metrics.
group_parasternal = merged_df[merged_df["usable_lead"] == "parasternal"]

# ---------------------------
# Part 4: Compute summary metrics for each group
# ---------------------------
# For group_diaphragm, using diaphragm metrics.
fields_diaphragm = [
    "wasted_effort_diaphragm",
    "diaphragm_induced_triggers",
    "auto_triggers",
    "forced_breath",
    "forced_breath_with_diaphragm",
    "n_diaphragm_true_peaks",
    "vent_onsets"
]
sums_diaphragm = { field: group_diaphragm[field].sum() for field in fields_diaphragm }
total_breaths_diaphragm = sums_diaphragm["vent_onsets"] + sums_diaphragm["wasted_effort_diaphragm"]

# For group_parasternal, using parasternal metrics.
fields_parasternal = [
    "wasted_effort_parasternal",
    "parasternal_induced_triggers",
    "auto_triggers",
    "forced_breath",
    "forced_breath_with_parasternal",
    "n_parasternal_true_peaks",
    "vent_onsets"
]
sums_parasternal = { field: group_parasternal[field].sum() for field in fields_parasternal }
total_breaths_parasternal = sums_parasternal["vent_onsets"] + sums_parasternal["wasted_effort_parasternal"]

# Calculate percentages relative to ventilator onsets.
pct_triggers_diaphragm = (100 * sums_diaphragm["diaphragm_induced_triggers"] / sums_diaphragm["vent_onsets"]) if sums_diaphragm["vent_onsets"] > 0 else 0
pct_wasted_diaphragm = (100 * sums_diaphragm["wasted_effort_diaphragm"] / total_breaths_diaphragm) if total_breaths_diaphragm > 0 else 0

pct_triggers_parasternal = (100 * sums_parasternal["parasternal_induced_triggers"] / sums_parasternal["vent_onsets"]) if sums_parasternal["vent_onsets"] > 0 else 0
pct_wasted_parasternal = (100 * sums_parasternal["wasted_effort_parasternal"] / total_breaths_parasternal) if total_breaths_parasternal > 0 else 0

# ---------------------------
# Part 5: Print the two separate asynchrony analysis summaries along with counts.
# ---------------------------
# For group_diaphragm (using diaphragm metrics)
num_meas_diaphragm = len(group_diaphragm)
num_patients_diaphragm = group_diaphragm["patient_id"].nunique()

print("\n=== Diaphragm Asynchrony Analysis (using diaphragm metrics) ===")
print(f"Measurements included: {num_meas_diaphragm}")
print(f"Unique patients: {num_patients_diaphragm}")
print(f"Total breaths(ventilator onset + wasted efforts): {total_breaths_diaphragm}")
print(f"Number of ventilator-onsets: {sums_diaphragm['vent_onsets']}")
print(f"Number of diaphragm true peaks: {sums_diaphragm['n_diaphragm_true_peaks']}")
print(f"Wasted Effort Diaphragm: {sums_diaphragm['wasted_effort_diaphragm']} ({pct_wasted_diaphragm:.2f}% of total breaths)")
print(f"Diaphragm induced triggers: {sums_diaphragm['diaphragm_induced_triggers']} ({pct_triggers_diaphragm:.2f}% of ventilator onsets)")
print(f"Auto triggers: {sums_diaphragm['auto_triggers']}")
print(f"True forced breath: {sums_diaphragm['forced_breath']}")
print(f"Forced Breath with Diaphragm activity: {sums_diaphragm['forced_breath_with_diaphragm']}")



# For group_parasternal (using parasternal metrics)
num_meas_parasternal = len(group_parasternal)
num_patients_parasternal = group_parasternal["patient_id"].nunique()

print("\n=== Parasternal Asynchrony Analysis (using parasternal metrics) ===")
print(f"Measurements included: {num_meas_parasternal}")
print(f"Unique patients: {num_patients_parasternal}")
print(f"Total breaths(ventilator onsets + wasted efforts): {total_breaths_parasternal}")
print(f"Number of ventilator-onsets: {sums_parasternal['vent_onsets']}")
print(f"Number of parasternal true peaks: {sums_parasternal['n_parasternal_true_peaks']}")
print(f"Wasted Effort Parasternal: {sums_parasternal['wasted_effort_parasternal']} ({pct_wasted_parasternal:.2f}% of total breaths)")
print(f"Parasternal induced triggers: {sums_parasternal['parasternal_induced_triggers']} ({pct_triggers_parasternal:.2f}% of ventilator onsets)")
print(f"Auto triggers: {sums_parasternal['auto_triggers']}")
print(f"True forced breath: {sums_parasternal['forced_breath']}")
print(f"Forced Breath with Parasternal activity: {sums_parasternal['forced_breath_with_parasternal']}")





In [ ]:
#STATISTICAL ANALYSIS ALL PATIENTS
import pandas as pd
from scipy.stats import binomtest

# === Bestand inladen ===
df = pd.read_csv("breathing_sync_summary_all_patients.csv")

# === Container voor resultaten ===
significance_results = []

# === Itereer over elke rij en toets of overlap significant is ===
for _, row in df.iterrows():
    n_dia = row["n_diaphragm_true_peaks"]
    overlap = row["overlap_diaphragm_vs_parasternal"]

    if n_dia == 0:
        p_value = None
        significant = None
    else:
        # Binomiale toets: H0 = overlap toevalskans 0.5
        p_value = binomtest(overlap, n=n_dia, p=0.5, alternative="greater").pvalue
        significant = p_value < 0.05

    significance_results.append({
        "patient_id": row["patient_id"],
        "timepoint": row["timepoint"],
        "n_diaphragm_true_peaks": n_dia,
        "overlap_with_parasternal": overlap,
        "overlap_percentage": round((overlap / n_dia * 100) if n_dia else 0, 1),
        "p_value": p_value,
        "significant": significant
    })

# === Resultaten tonen ===
sig_df = pd.DataFrame(significance_results)
sig_df = sig_df.sort_values(by=["patient_id", "timepoint"])
print(sig_df)

# === Optioneel: opslaan ===
sig_df.to_csv("significance_overlap_diaphragm_vs_parasternal.csv", index=False)
print("\n✅ Results stored in 'significance_overlap_diaphragm_vs_parasternal.csv'")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === CSV-bestand inladen ===
df = pd.read_csv("significance_overlap_diaphragm_vs_parasternal.csv")

# === Pivot tabellen maken ===
pivot_vals = df.pivot(index="patient_id", columns="timepoint", values="overlap_percentage")
pivot_sig = df.pivot(index="patient_id", columns="timepoint", values="significant")

# === Heatmap tekenen ===
plt.figure(figsize=(12, 6))
ax = sns.heatmap(pivot_vals, annot=True, fmt=".1f", cmap="YlGnBu",
                 linewidths=0.5, linecolor='gray', cbar_kws={"label": "Overlap (%)"})

# === Randen tekenen voor significantie ===
for y in range(pivot_vals.shape[0]):
    for x in range(pivot_vals.shape[1]):
        is_sig = pivot_sig.iloc[y, x]
        if pd.notna(is_sig) and is_sig:
            ax.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))

# === Opmaak ===
plt.title("Overlap diaphragm ↔ parasternal EMG-peaks (significant results marked red)")
plt.xlabel("Time")
plt.ylabel("Patient ID")
plt.tight_layout()
plt.show()


In [ ]:
# STATISTISCHE TOETS: parasternal overlap met diaphragm
import pandas as pd
from scipy.stats import binomtest
import matplotlib.pyplot as plt
import seaborn as sns

# === Data inladen ===
df = pd.read_csv("breathing_sync_summary_all_patients.csv")

# === Resultaten verzamelen ===
results = []

for _, row in df.iterrows():
    n_int = row["n_parasternal_true_peaks"]
    overlap = row["overlap_parasternal_vs_diaphragm"]

    if n_int == 0:
        p_value = None
        significant = None
        perc = 0
    else:
        p_value = binomtest(overlap, n=n_int, p=0.5, alternative="greater").pvalue
        significant = p_value < 0.05
        perc = round((overlap / n_int) * 100, 1)

    results.append({
        "patient_id": row["patient_id"],
        "timepoint": row["timepoint"],
        "n_parasternal_true_peaks": n_int,
        "overlap_with_diaphragm": overlap,
        "overlap_percentage": perc,
        "p_value": p_value,
        "significant": significant
    })

# === Dataframe bouwen en opslaan ===
sig_df = pd.DataFrame(results)
sig_df = sig_df.sort_values(by=["patient_id", "timepoint"])
sig_df.to_csv("significance_overlap_parasternal_vs_diaphragm.csv", index=False)
print("\n✅ Results stored in 'significance_overlap_parasternal_vs_diaphragm.csv'")

# === Heatmap genereren ===
pivot_vals = sig_df.pivot(index="patient_id", columns="timepoint", values="overlap_percentage")
pivot_sig = sig_df.pivot(index="patient_id", columns="timepoint", values="significant")

plt.figure(figsize=(12, 6))
ax = sns.heatmap(pivot_vals, annot=True, fmt=".1f", cmap="YlGnBu",
                 linewidths=0.5, linecolor='gray', cbar_kws={"label": "Overlap (%)"})

# Rode rand tekenen bij significante cellen
for y in range(pivot_vals.shape[0]):
    for x in range(pivot_vals.shape[1]):
        is_sig = pivot_sig.iloc[y, x]
        if pd.notna(is_sig) and is_sig:
            ax.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))

plt.title("Overlap parasternal ↔ diaphragm EMG-peaks (significant results marked red)")
plt.xlabel("Time")
plt.ylabel("Patient ID")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import binomtest

# ---------------------------
# Part 1: Load measurement classification and select included measurements
# ---------------------------
msr_df = pd.read_csv("measurement_classification.csv")
# Create a key by concatenating "Patient ID" and "Time ID" (after stripping spaces)
msr_df["key"] = msr_df["Patient ID"].astype(str).str.strip() + "_" + msr_df["Time ID"].astype(str).str.strip()
# Retain only measurements labeled as "Include"
include_df = msr_df[msr_df["Label"] == "Include"]
included_keys = set(include_df["key"].unique())

# ---------------------------
# Part 2: Load asynchrony analysis data and filter for included measurements
# ---------------------------
ana_df = pd.read_csv("asynchrony_analysis_updated.csv")
ana_df["key"] = ana_df["patient_id"].astype(str).str.strip() + "_" + ana_df["timepoint"].astype(str).str.strip()
included_ana = ana_df[ana_df["key"].isin(included_keys)].copy()

# ---------------------------
# Part 3: Compute overall overlap summaries
# ---------------------------
# For diaphragm overlap with parasternal:
#   n_diaphragm_true_peaks (total diaphragm peaks) and dia_int_overlap (overlap count)
total_diaphragm_true = included_ana["n_diaphragm_true_peaks"].sum()
overlap_diaphragm = included_ana["dia_int_overlap"].sum()
perc_diaphragm = (overlap_diaphragm / total_diaphragm_true * 100) if total_diaphragm_true > 0 else 0

# For parasternal overlap with diaphragm:
#   n_parasternal_true_peaks (total parasternal peaks) and int_dia_overlap (overlap count)
total_parasternal_true = included_ana["n_parasternal_true_peaks"].sum()
overlap_parasternal = included_ana["int_dia_overlap"].sum()
perc_parasternal = (overlap_parasternal / total_parasternal_true * 100) if total_parasternal_true > 0 else 0

print("=== Overall Overlap Summary for Included Measurements ===\n")
print("Diaphragm with Parasternal:")
print(f"  Total diaphragm true peaks: {total_diaphragm_true}")
print(f"  Overlap (diaphragm with parasternal): {overlap_diaphragm} ({perc_diaphragm:.1f}%)\n")
print("Parasternal with Diaphragm:")
print(f"  Total parasternal true peaks: {total_parasternal_true}")
print(f"  Overlap (parasternal with diaphragm): {overlap_parasternal} ({perc_parasternal:.1f}%)\n")

# ---------------------------
# Part 4: Compute per-measurement overlap percentages and p-values (using a binomial test)
# ---------------------------
# For diaphragm overlap: p-value testing if observed proportion (dia_int_overlap / n_diaphragm_true_peaks) > 0.5
def compute_p_diaphragm(row):
    n = row["n_diaphragm_true_peaks"]
    overlap = row["dia_int_overlap"]
    if n > 0:
        return binomtest(int(overlap), n=int(n), p=0.5, alternative="greater").pvalue
    else:
        return None

included_ana["p_value_diaphragm"] = included_ana.apply(compute_p_diaphragm, axis=1)

# For parasternal overlap: test if (int_dia_overlap / n_parasternal_true_peaks) > 0.5
def compute_p_parasternal(row):
    n = row["n_parasternal_true_peaks"]
    overlap = row["int_dia_overlap"]
    if n > 0:
        return binomtest(int(overlap), n=int(n), p=0.5, alternative="greater").pvalue
    else:
        return None

included_ana["p_value_parasternal"] = included_ana.apply(compute_p_parasternal, axis=1)

# Also compute per-measurement overlap percentages
included_ana["overlap_percentage_diaphragm"] = included_ana.apply(
    lambda row: (row["dia_int_overlap"] / row["n_diaphragm_true_peaks"] * 100)
    if row["n_diaphragm_true_peaks"] > 0 else None, axis=1
)

included_ana["overlap_percentage_parasternal"] = included_ana.apply(
    lambda row: (row["int_dia_overlap"] / row["n_parasternal_true_peaks"] * 100)
    if row["n_parasternal_true_peaks"] > 0 else None, axis=1
)

# ---------------------------
# Part 5: Create pivot tables (visual dataframes) for the percentages and p-values
# ---------------------------
pivot_diaphragm = included_ana.pivot(index="patient_id", columns="timepoint", values="overlap_percentage_diaphragm")
pivot_parasternal = included_ana.pivot(index="patient_id", columns="timepoint", values="overlap_percentage_parasternal")

pivot_diaphragm_p = included_ana.pivot(index="patient_id", columns="timepoint", values="p_value_diaphragm")
pivot_parasternal_p = included_ana.pivot(index="patient_id", columns="timepoint", values="p_value_parasternal")

# ---------------------------
# Part 6: Print pivot tables and produce heatmaps with red boxes for significant cells (p < 0.05)
# ---------------------------
print("=== Pivot Table: Overlap Percentage (Diaphragm with Parasternal) ===")
print(pivot_diaphragm)
print("\n=== Pivot Table: Overlap Percentage (Parasternal with Diaphragm) ===")
print(pivot_parasternal)

# Heatmap for diaphragm overlap
plt.figure(figsize=(12, 6))
ax1 = sns.heatmap(pivot_diaphragm, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=0.5,
                  linecolor='gray', cbar_kws={"label": "Overlap (%)"})
plt.title("Heatmap: Diaphragm with Parasternal Overlap (%)")
plt.xlabel("Time")
plt.ylabel("Patient ID")
# Draw red rectangle around cells where p < 0.05 (based on pivot_diaphragm_p)
for y in range(pivot_diaphragm.shape[0]):
    for x in range(pivot_diaphragm.shape[1]):
        p_val = pivot_diaphragm_p.iloc[y, x]
        if pd.notna(p_val) and p_val < 0.05:
            ax1.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))
plt.tight_layout()
plt.show()

# Heatmap for parasternal overlap
plt.figure(figsize=(12, 6))
ax2 = sns.heatmap(pivot_parasternal, annot=True, fmt=".1f", cmap="YlGnBu", linewidths=0.5,
                  linecolor='gray', cbar_kws={"label": "Overlap (%)"})
plt.title("Heatmap: Parasternal with Diaphragm Overlap (%)")
plt.xlabel("Time")
plt.ylabel("Patient ID")
# Draw red rectangle where p < 0.05 (using pivot_parasternal_p)
for y in range(pivot_parasternal.shape[0]):
    for x in range(pivot_parasternal.shape[1]):
        p_val = pivot_parasternal_p.iloc[y, x]
        if pd.notna(p_val) and p_val < 0.05:
            ax2.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Laad de data (pas het pad aan indien nodig)
df = pd.read_csv("emg_peak_assessment_all_patients.csv")

# Filter alleen true peaks
true_peaks = df[df['label'] == 'True peak']

# Groepeer per patiënt en timepoint
grouped = true_peaks.groupby(['patient_id', 'timepoint'])

results = []

# Voor elke patiënt en timepoint
for (pid, tp), group in grouped:
    # Splits op kanaal
    diaphragm_peaks = group[group['channel'] == 'diaphragm']['time_s'].values
    parasternal_peaks = group[group['channel'] == 'parasternal']['time_s'].values

    # Als één van beide leeg is, kunnen we geen overeenkomst controleren
    if len(diaphragm_peaks) == 0 or len(parasternal_peaks) == 0:
        results.append({
            'patient_id': pid,
            'timepoint': tp,
            'chi2': np.nan,
            'p_value': np.nan,
            'reject_null': 'Insufficient data'
        })
        continue

    # Check of er binnen 0.5s een piek is in het andere kanaal
    diaphragm_matched = sum(
        any(abs(dp - pp) <= 0.5 for pp in parasternal_peaks) for dp in diaphragm_peaks
    )
    diaphragm_unmatched = len(diaphragm_peaks) - diaphragm_matched

    parasternal_matched = sum(
        any(abs(pp - dp) <= 0.5 for dp in diaphragm_peaks) for pp in parasternal_peaks
    )
    parasternal_unmatched = len(parasternal_peaks) - parasternal_matched

    # Contingency table en chi-kwadraat test
    contingency = np.array([
        [diaphragm_matched, diaphragm_unmatched],
        [parasternal_matched, parasternal_unmatched]
    ])

    try:
        chi2, p, _, _ = chi2_contingency(contingency)
        reject_null = 'Yes' if p < 0.05 else 'No'
    except ValueError:
        chi2, p, reject_null = np.nan, np.nan, 'Invalid test'

    results.append({
        'patient_id': pid,
        'timepoint': tp,
        'chi2': chi2,
        'p_value': p,
        'reject_null': reject_null
    })

# Zet om naar dataframe en sla op als CSV
results_df = pd.DataFrame(results)
results_df.to_csv("emg_peak_match_stats.csv", index=False)

print("Analysis complete results stored in 'emg_peak_match_stats.csv'.")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Laad de resultaten van de analyse
results_df = pd.read_csv("emg_peak_match_stats.csv")

# Zet data in een pivot-formaat voor heatmap
pivot_chi2 = results_df.pivot(index='patient_id', columns='timepoint', values='chi2')
pivot_pval = results_df.pivot(index='patient_id', columns='timepoint', values='p_value')

# Plot de heatmap met chi-kwadraatwaarden
plt.figure(figsize=(12, 8))
ax = sns.heatmap(pivot_chi2, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'Chi-kwadraatwaarde'})

# Rood kader tekenen rond significante p-values (< 0.05)
for y in range(pivot_chi2.shape[0]):
    for x in range(pivot_chi2.shape[1]):
        pval = pivot_pval.iloc[y, x]
        if pd.notna(pval) and pval < 0.05:
            ax.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))

# Aanpassingen voor leesbaarheid
plt.title('Heatmap Chi-squared test EMG overlap')
plt.xlabel('Time')
plt.ylabel('Patient ID')

# Legenda toevoegen voor rood kader
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='none', edgecolor='red', linewidth=2, label='p < 0.05 (significant)')]
plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1))

plt.tight_layout()
plt.show()

Correlation Ventilator with EMG

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# === Stap 1: Laad de data ===
emg_df = pd.read_csv("emg_peak_assessment_all_patients.csv")
vent_df = pd.read_csv("ventilator_breaths_all_patients.csv")

# Filter op true peaks
true_peaks = emg_df[emg_df['label'] == 'True peak']
vent_groups = vent_df.groupby(['patient_id', 'timepoint'])

# === Stap 2: Functie voor chi-kwadraat analyse per kanaal ===
def compute_chi2_vs_ventilator(emg_channel):
    results = []
    grouped_emg = true_peaks[true_peaks['channel'] == emg_channel].groupby(['patient_id', 'timepoint'])

    for (pid, tp), emg_group in grouped_emg:
        emg_peaks = emg_group['time_s'].values
        try:
            vent_onsets = vent_groups.get_group((pid, tp))['time_onset_s'].values
        except KeyError:
            results.append({
                'patient_id': pid,
                'timepoint': tp,
                'chi2': np.nan,
                'p_value': np.nan,
                'reject_null': 'Insufficient ventilator data'
            })
            continue

        if len(emg_peaks) == 0 or len(vent_onsets) == 0:
            results.append({
                'patient_id': pid,
                'timepoint': tp,
                'chi2': np.nan,
                'p_value': np.nan,
                'reject_null': 'Insufficient data'
            })
            continue

        emg_matched = sum(any(abs(ep - vo) <= 0.3 for vo in vent_onsets) for ep in emg_peaks)
        emg_unmatched = len(emg_peaks) - emg_matched

        vent_matched = sum(any(abs(vo - ep) <= 0.3 for ep in emg_peaks) for vo in vent_onsets)
        vent_unmatched = len(vent_onsets) - vent_matched

        contingency = np.array([
            [emg_matched, emg_unmatched],
            [vent_matched, vent_unmatched]
        ])

        try:
            chi2, p, _, _ = chi2_contingency(contingency)
            reject_null = 'Yes' if p < 0.05 else 'No'
        except ValueError:
            chi2, p, reject_null = np.nan, np.nan, 'Invalid test'

        results.append({
            'patient_id': pid,
            'timepoint': tp,
            'chi2': chi2,
            'p_value': p,
            'reject_null': reject_null
        })

    return pd.DataFrame(results)

# === Stap 3: Analyse uitvoeren en opslaan ===
df_chi2_diaphragm = compute_chi2_vs_ventilator('diaphragm')
df_chi2_parasternal = compute_chi2_vs_ventilator('parasternal')

df_chi2_diaphragm.to_csv("chi2_emg_diaphragm_vs_ventilator.csv", index=False)
df_chi2_parasternal.to_csv("chi2_emg_parasternal_vs_ventilator.csv", index=False)

# === Stap 4: Heatmap functie ===
def plot_chi2_heatmap(df, title):
    # Timepoints die we willen meenemen in logische (chronologische) volgorde
    valid_order = ['t0', 't0v1', 't1', 't3', 't4', 't6', 't12', 't24', 't48']
    df = df[df['timepoint'].isin(valid_order)]

    # Zet de timepoints als categorische data zodat ze correct gesorteerd worden
    df['timepoint'] = pd.Categorical(df['timepoint'], categories=valid_order, ordered=True)

    # Pivot tabellen voor heatmap en p-waarden
    pivot_chi2 = df.pivot(index='patient_id', columns='timepoint', values='chi2').sort_index(axis=1)
    pivot_pval = df.pivot(index='patient_id', columns='timepoint', values='p_value').reindex(columns=pivot_chi2.columns)

    # Plot heatmap
    plt.figure(figsize=(12, 8))
    ax = sns.heatmap(pivot_chi2, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'Chi-squared value'})

    # Rode rand tekenen voor significante p-waarden
    for y in range(pivot_chi2.shape[0]):
        for x in range(pivot_chi2.shape[1]):
            pval = pivot_pval.iloc[y, x]
            if pd.notna(pval) and pval < 0.05:
                ax.add_patch(plt.Rectangle((x, y), 1, 1, fill=False, edgecolor='red', lw=2))

    # Labels en legenda
    plt.title(title)
    plt.xlabel('Timepoint')
    plt.ylabel('Patiënt ID')

    legend_elements = [Patch(facecolor='none', edgecolor='red', linewidth=2, label='p < 0.05 (significant)')]
    plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1))

    plt.tight_layout()
    plt.show()

# Voor diaphragm-resultaten
plot_chi2_heatmap(df_chi2_diaphragm, 'Chi-squared: EMG Diaphragm vs Ventilator Onsets')

# Voor parasternal-resultaten
plot_chi2_heatmap(df_chi2_parasternal, 'Chi-squared: EMG Parasternal vs Ventilator Onsets')

In [ ]:
#WILCOXON SIGNED RANK TEST
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon

# Laad data
df = pd.read_csv("breathing_sync_summary_all_patients.csv")

# Bereken totale wasted efforts per spier
df["total_diaphragm"] = df["wasted_effort_diaphragm"] + df["complete_wasted_effort"]
df["total_parasternal"] = df["wasted_effort_parasternal"] + df["complete_wasted_effort"]

# Wilcoxon signed-rank test (vergelijking tussen de twee spieren)
stat, p_value = wilcoxon(df["total_diaphragm"], df["total_parasternal"])

print("Wilcoxon signed-rank test:")
print(f"  W-statistic: {stat}")
print(f"  p-value: {p_value:.5f}")
if p_value < 0.05:
    print("  ✅ Significant difference found between diaphragm and parasternal.")
else:
    print("  ❌ No significant difference found.")

# 📈 Visualisatie: boxplot
data_melted = df[["total_diaphragm", "total_parasternal"]].melt(var_name="Lead", value_name="Number of Wasted Efforts")

plt.figure(figsize=(8, 5))
sns.boxplot(data=data_melted, x="Lead", y="Number of Wasted Efforts", palette="pastel")
sns.swarmplot(data=data_melted, x="Lead", y="Number of Wasted Efforts", color="black", alpha=0.6, size=3)
plt.title("Wasted efforts diaphragm vs parasternal")
plt.ylabel("Number of wasted efforts")
plt.xlabel("Lead")
plt.grid(True, axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel

# Laad data
df = pd.read_csv("breathing_sync_summary_all_patients.csv")

# Totale wasted efforts berekenen
df["total_diaphragm"] = df["wasted_effort_diaphragm"] + df["complete_wasted_effort"]
df["total_parasternal"] = df["wasted_effort_parasternal"] + df["complete_wasted_effort"]

# Paired Student’s t-test
t_stat, p_value = ttest_rel(df["total_diaphragm"], df["total_parasternal"])

print("Paired Student’s t-test:")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_value:.5f}")
if p_value < 0.05:
    print("  ✅ Significant difference found between diaphragm and parasternal.")
else:
    print("  ❌ No significant difference found")

# 📈 Boxplot visualisatie
data_melted = df[["total_diaphragm", "total_parasternal"]].melt(
    var_name="Lead", value_name="Number of Wasted Efforts"
)

plt.figure(figsize=(8, 5))
sns.boxplot(data=data_melted, x="Lead", y="Number of Wasted Efforts", palette="pastel")
sns.swarmplot(data=data_melted, x="Lead", y="Number of Wasted Efforts", color="black", alpha=0.6, size=3)
plt.title("Wasted efforts diaphragm vs parasternal")
plt.ylabel("Number of wasted efforts")
plt.xlabel("Lead")
plt.grid(True, axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()